***
***

<img width='700' src="https://user-images.githubusercontent.com/8030363/108961534-b9a66980-7634-11eb-96e2-cc46589dcb8c.png" style="vertical-align:middle">

## Pre-Knowledge Graph Build Data Preparation
***

**Author:** [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**
  
<br>  
  
**Purpose:** This notebook serves as a script to download and process data in order to generate mapping and filtering data needed to build edges for the PheKnowLator knowledge graph. For more information on the data sources utilize within this script, please see the [Data Sources](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources) Wiki page.

<br>

**Assumptions:**   
- Raw data downloads ➞ `./resources/processed_data/unprocessed_data`    
- Processed data write location ➞ `./resources/processed_data`  

<br>

**Dependencies:**   
- **Scripts**: This notebook utilizes several helper functions, which are stored in the [`data_utils.py`](https://github.com/callahantiff/PheKnowLator/blob/master/pkt_kg/utils/data_utils.py) and [`kg_utils.py`](https://github.com/callahantiff/PheKnowLator/blob/master/pkt_kg/utils/kg_utils.py) scripts.  
- **Data**: Hyperlinks to all downloaded and generated data sources are provided through [Zenodo](https://zenodo.org/doi/10.5281/zenodo.7030039). <u>This notebook will download everything that is needed for you</u>.  
_____
***

## Table of Contents
***

### [Create Identifier Maps ](#create-identifier-maps)  
- [HUMAN TRANSCRIPT, GENE, AND PROTEIN IDENTIFIER MAPPING](#human-transcript,-gene,-and-protein-identifier-mapping)
  - [Entrez Gene-Ensembl Transcript](#entrezgene-ensembltranscript)  
  - [Entrez Gene-Protein Ontology](#entrezgene-proteinontology)  
  - [Ensembl Gene-Entrez Gene](#ensemblgene-entrezgene)
  - [Gene Symbol-Ensembl Transcript](#genesymbol-ensembltranscript)  
  - [STRING-Protein Ontology](#string-proteinontology)  
  - [Uniprot Accession-Protein Ontology](#uniprotaccession-proteinontology)
  

- [OTHER IDENTIFIER MAPPING](#other-identifier-mapping) 
  - [ChEBI Identifiers](#mesh-chebi) 
  - [Human Disease and Phenotype Identifiers](#disease-identifiers)
  - [Human Protein Atlas Tissue and Cell Types](#hpa-uberon)  
  - [Reactome Pathways - Pathway Ontology](#reactome-pw)  
  - [Genomic Identifiers - Sequence Ontology](#genomic-soo)  


### [Create Edge Datasets](#create-edge-datasets)
- [ONTOLOGIES](#ontologies)  
  - [Protein Ontology](#protein-ontology)  
  - [Relations Ontology](#relations-ontology)  


- [LINKED DATA](#linked-data)  
  - [Clinvar Variant-Diseases and Phenotypes](#clinvar-variant)
  - [Uniprot Protein-Cofactor and Protein-Catalyst](#uniprot-protein-cofactorcatalyst)  


### [Create Instance Data and/or Subclass Metadata](#create-instance-metadata)  
- [Genes/RNA](#gene-and-rna-metadata)
- [Pathways](#pathway-metadata)
- [Variants](#variant-metadata) 
- [Relations](#relations-metadata) 

____

<br>

## Set-Up Environment
_____

In [ ]:
# # uncomment and run to install any required modules from notebooks/requirements.txt
# import sys
# !{sys.executable} -m pip install -r requirements.txt

In [ ]:
# if running a local version of pkt_kg, uncomment the code below
# import sys
# sys.path.append('../')

In [ ]:
# import needed libraries
import datetime
import glob
import itertools
import networkx
import numpy
import os
import openpyxl
import pandas
import pickle
import re
import requests
import sys

from collections import Counter
from functools import reduce
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import OWL, RDF, RDFS
from reactome2py import content
from tqdm import tqdm
from typing import Dict

from pkt_kg.utils import *  # import pkt_kg utility script containing helper functions

#### Define Global Variables

In [ ]:
# directory to use for processing data
unprocessed_data_location = '../resources/processed_data/unprocessed_data/'
processed_data_location = '../resources/processed_data/'

# directory to write relations data to
relations_data_location = '../resources/relations_data/'

# directory to write node metadata to
node_data_location = '../resources/node_data/'

# directory to write kg construction approach dictionary to
construction_approach_location = '../resources/construction_approach/'

# directory to write ontology data to
ontology_data_location = '../resources/ontologies/'

# owltools location
owltools_location = '../pkt_kg/libs/owltools'

# obo spacespace
obo = Namespace('http://purl.obolibrary.org/obo/')

<br>

***
***
### CREATE MAPPING DATASETS  <a class="anchor" id="create-identifier-maps"></a>
***
***

### Human Transcript, Gene, and Protein Identifier Mapping  <a class="anchor" id="human-transcript,-gene,-and-protein-identifier-mapping"></a>
***

**Data Source Wiki Pages:**   
- [Ensembl](https://uswest.ensembl.org/)  
- [Uniprot Knowledgebase](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#uniprot-knowledgebase)  
- [HGNC](ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt) 
- [NCBI Gene](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#ncbi-gene) 
- [Protein Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#protein-ontology)

<br>

**Purpose:** To map create `protein-coding gene`-`protein` edges and mappings between the identifiers types listed below. The edges types produced from each of these mappings will be further described within each of the subsequent identifier mapping sections:  
- [Entrez Gene-Ensembl Transcript](#entrezgene-ensembltranscript)  
- [Entrez Gene-Protein Ontology](#entrezgene-proteinontology)  
- [Ensembl Gene-Entrez Gene](#ensemblgene-entrezgene)
- [Gene Symbol-Ensembl Transcript](#genesymbol-ensembltranscript)  
- [STRING-Protein Ontology](#string-proteinontology)  
- [Uniprot Accession-Protein Ontology](#uniprotaccession-proteinontology)

<br>

**Gene and Transcript Types:** The transcript and gene/locus types were reviewed by a PhD Molecular biologist to confirm whether or not they should be classified as `protein-coding` or not, which is useful for creating `genomic`-`rna`, `genomic`-`protein`, and `rna`-`protein` edges in the knowledge graph. For more information on this classification, please see the table below. Definitions of concepts in the table have been taken from [HGNC](https://www.genenames.org/help/symbol-report/), [Ensembl](https://uswest.ensembl.org/info/genome/genebuild/biotypes.html), [NCBI](https://www.ncbi.nlm.nih.gov/IEB/ToolBox/CPP_DOC/lxr/source/src/objects/entrezgene/entrezgene.asn), and Wikipedia.

<table>
<th align="center">Gene and Transcript Type</th>  
<th align="center">Definition</th>
<th align="center">Type</th>
<th align="center">Genomic material <i>transcribed_to</i> RNA</th>
<th align="center">RNA <i>translated_to</i> Protein</th>
<th align="center">Genomic material <i>has_gene_product</i> Protein</th>
<tr>
  <td rowspan="2">biological-region</td> 
  <td rowspan="2">Biological_region (SO:0001411); Special note: This is a parental feature spanning all other feature annotation on each RefSeq Functional Element record. It is a 'misc_feature' in GenBank flat files but a 'Region' feature in ASN.1 and GFF3 formats</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
  <td>transcript</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td> 
</tr>
<tr>
  <td rowspan="2">IG_C_gene</td> 
  <td rowspan="2">Constant chain immunoglobulin gene that undergoes somatic recombination before transcription</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
  <td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">IG_C_pseudogene</td> 
  <td rowspan="2">Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>	 	 
<tr>
  <td rowspan="2">IG_D_gene</td> 
  <td rowspan="2">Diversity chain immunoglobulin gene that undergoes somatic recombination before transcription</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">IG_J_gene</td> 
  <td rowspan="2">IG J gene: Joining chain immunoglobulin gene that undergoes somatic recombination before transcription</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">IG_J_pseudogene</td> 
  <td rowspan="2">Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">IG_pseudogene</td> 
  <td rowspan="2">Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">IG_V_gene</td> 
  <td rowspan="2">Variable chain immunoglobulin gene that undergoes somatic recombination before transcription</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">IG_V_pseudogene</td> 
  <td rowspan="2">Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">lncRNA</td> 
  <td rowspan="2">RNA, long non-coding - non-protein coding genes that encode long non-coding RNAs (lncRNAs) (SO:0001877); these are at least 200 nt in length. Subtypes include intergenic (SO:0001463), intronic (SO:0001903) and antisense (SO:0001904)</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">miRNA</td> 
  <td rowspan="2">RNA, micro - non-protein coding genes that encode microRNAs (miRNAs) (SO:0001265)</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">misc_RNA</td> 
  <td rowspan="2">Non-protein coding genes that encode miscellaneous types of small ncRNAs, such as vault (SO:0000404) and Y (SO:0000405) RNA genes</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">Mt_rRNA</td> 
  <td rowspan="2">Mitochondrial rRNA</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">Mt_tRNA</td> 
  <td rowspan="2">Mitochondrial tRNA</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">ncRNA</td> 
  <td rowspan="2">Noncoding RNA</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td> 
</tr>
<tr>
  <td rowspan="2">non_stop_decay</td> 
  <td rowspan="2">Transcripts that have polyA features (including signal) without a prior stop codon in the CDS, i.e. a non-genomic polyA tail attached directly to the CDS without 3' UTR. These transcripts are subject to degradation</td>
  <td>gene</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">nonsense_mediated_decay</td> 
  <td rowspan="2">If the coding sequence (following the appropriate reference) of a transcript finishes >50bp from a downstream splice site then it is tagged as NMD. If the variant does not cover the full reference coding sequence then it is annotated as NMD if NMD is unavoidable i.e. no matter what the exon structure of the missing portion is the transcript will be subject to NMD</td>
  <td>gene</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">other</td> 
  <td rowspan="2">other</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">phenotype</td> 
  <td rowspan="2"> Mapped phenotypes where the causative gene has not been identified (SO:0001500) </td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td> 
</tr>
<tr>
  <td rowspan="2">polymorphic_pseudogene</td> 
  <td rowspan="2">Pseudogene owing to a SNP/DIP but in other individuals/haplotypes/strains the gene is translated</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">processed_pseudogene</td> 
  <td rowspan="2">Pseudogene that lack introns and is thought to arise from reverse transcription of mRNA followed by reinsertion of DNA into the genome</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">processed_transcript</td> 
  <td rowspan="2">Gene/transcript that doesn't contain an open reading frame</td>
  <td>gene</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">protein_coding</td> 
  <td rowspan="2">Contains an open reading frame (ORF)</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>yes</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>yes</td> 
</tr>
<tr>
  <td rowspan="2">pseudogene</td> 
  <td rowspan="2">Have homology to proteins but generally suffer from a disrupted coding sequence and an active homologous gene can be found at another locus. Sometimes these entries have an intact coding sequence or an open but truncated open reading frame, in which case there is other evidence used (for example genomic polyA stretches at the 3' end) to classify them as a pseudogene</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">retained_intron</td> 
  <td rowspan="2">Has an alternatively spliced transcript believed to contain intronic sequence relative to other, coding, variants</td>
  <td>gene</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">ribozyme</td> 
  <td rowspan="2">Ribozymes are RNA molecules that have the ability to catalyze specific biochemical reactions, including RNA splicing in gene expression, similar to the action of protein enzymes</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">rRNA</td> 
  <td rowspan="2">RNA, ribosomal - non-protein coding genes that encode ribosomal RNAs (rRNAs) (SO:0001637)</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">rRNA_pseudogene</td> 
  <td rowspan="2">A gene that has homology to known protein-coding genes but contain a frameshift and/or stop codon(s) which disrupts the open reading frame. Thought to have arisen through duplication followed by loss of function</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">scaRNA</td> 
  <td rowspan="2">Small Cajal body-specific RNAs are a class of small nucleolar RNAs that specifically localize to the Cajal body, a nuclear organelle involved in the biogenesis of small nuclear ribonucleoproteins/td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">scRNA</td> 
  <td rowspan="2">RNA, small cytoplasmic - non-protein coding genes that encode small cytoplasmic RNAs (scRNAs) (SO:0001266)</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">snoRNA</td> 
  <td rowspan="2">RNA, small nucleolar - non-protein coding genes that encode small nucleolar RNAs (snoRNAs) containing C/D or H/ACA box domains (SO:0001267)</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
  <td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">snRNA</td> 
  <td rowspan="2">RNA, small nuclear - non-protein coding genes that encode small nuclear RNAs (snRNAs) (SO:0001268)</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">sRNA</td> 
  <td rowspan="2">Bacterial small RNAs (sRNA) are small RNAs produced by bacteria; they are 50- to 500-nucleotide non-coding RNA molecules, highly structured and containing several stem-loops</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">TEC</td> 
  <td rowspan="2">TEC (To be Experimentally Confirmed). This is used for non-spliced EST clusters that have polyA features. This category has been specifically created for the ENCODE project to highlight regions that could indicate the presence of protein coding genes that require experimental validation, either by 5' RACE or RT-PCR to extend the transcripts, or by confirming expression of the putatively-encoded peptide with specific antibodies</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>yes</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">TR_C_gene</td> 
  <td rowspan="2">Constant chain T cell receptor gene that undergoes somatic recombination before transcription/td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">TR_D_gene</td> 
  <td rowspan="2">Diversity chain T cell receptor gene that undergoes somatic recombination before transcription/td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
  <td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">TR_J_gene</td> 
  <td rowspan="2">Joining chain T cell receptor gene that undergoes somatic recombination before transcription</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">TR_J_pseudogene</td> 
  <td rowspan="2">T cell receptor pseudogene - T cell receptor gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">TR_V_gene</td> 
  <td rowspan="2">Variable chain T cell receptor gene that undergoes somatic recombination before transcription</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">TR_V_pseudogene</td> 
  <td rowspan="2">T cell receptor pseudogene - T cell receptor gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">transcribed_processed_pseudogene</td> 
  <td rowspan="2">Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary'</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">transcribed_unitary_pseudogene</td> 
  <td rowspan="2">Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary'</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">transcribed_unprocessed_pseudogene</td> 
  <td rowspan="2">Pseudogene where protein homology or genomic structure indicates a pseudogene, but the presence of locus-specific transcripts indicates expression. These can be classified into 'Processed', 'Unprocessed' and 'Unitary'</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
<tr>
<td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">translated_processed_pseudogene</td> 
  <td rowspan="2">Pseudogenes that have mass spec data suggesting that they are also translated. These can be classified into 'Processed', 'Unprocessed'</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
  <td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">translated_unprocessed_pseudogene</td> 
  <td rowspan="2">Inactivated immunoglobulin gene. Immunoglobulin gene segments that are inactivated due to frameshift mutations and/or stop codons in the open reading frame</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
  <td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">tRNA</td> 
  <td rowspan="2">Transfer RNA</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
  <td>transcript</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td> 
</tr>
<tr>
  <td rowspan="2">unitary_pseudogene</td> 
  <td rowspan="2">A species specific unprocessed pseudogene without a parent gene, as it has an active orthologue in another species</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
  <td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2">unknown</td> 
  <td rowspan="2">Entries where the locus type is currently unknown</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
  <td>transcript</td> 
  <td> --- </td> 
  <td> --- </td> 
  <td> --- </td> 
</tr>
<tr>
  <td rowspan="2">unprocessed_pseudogene</td> 
  <td rowspan="2">Pseudogene that can contain introns since produced by gene duplication</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
  <td>transcript</td> 
  <td>yes</td> 
  <td>yes</td> 
  <td>no</td> 
</tr>
<tr>
  <td rowspan="2" align="center">vaultRNA</td> 
  <td rowspan="2" align="center">Short non coding RNA genes that form part of the vault ribonucleoprotein complex</td>
  <td>gene</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td>
</tr>
  <td>transcript</td> 
  <td>yes</td> 
  <td>no</td> 
  <td>no</td> 
</tr>
</table> 


<br>

**Output:** This script downloads and saves the following data:  
- Human Ensembl Gene Set ➞ `Homo_sapiens.GRCh38.<<release>>.gtf`
- Human Ensembl-UniProt Identifiers ➞ `Homo_sapiens.GRCh38.<<release>>.uniprot.tsv` 
- Human Ensembl-Entrez Identifiers ➞ `Homo_sapiens.GRCh38.<<release>>.entrez.tsv` 
- Human Gene Identifiers ➞ [`Homo_sapiens.gene_info`](ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz), [`hgnc_complete_set.txt`](ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt)  
- Human Protein Identifiers ➞ [`promapping.txt`](https://proconsortium.org/download/current/promapping.txt)  
- UniProt Identifiers ➞ [`uniprot_identifier_mapping.tab`](https://www.uniprot.org/uniprot/?query=&fil=organism%3A%22Homo%20sapiens%20(Human)%20%5B9606%5D%22&columns=id%2Cdatabase(GeneID)%2Cdatabase(Ensembl)%2Cdatabase(HGNC)%2Cgenes(PREFERRED)%2Cgenes(ALTERNATIVE))

*All Merged Data Sets:*  
- `Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt` 
- `Merged_gene_rna_protein_identifiers.pkl`  

***

**Genomic Typing Dictionary**  
Read in the  `genomic_typing_dict.pkl` dictionary, which is needed in order to preprocess the genomic identifier datasets.

In [ ]:
# download data
url = 'https://zenodo.org/records/10056198/files/genomic_typing_dict.pkl.zip?download=1'
if not os.path.exists(unprocessed_data_location + 'genomic_typing_dict.pkl'):
    data_downloader(url, unprocessed_data_location)

# load data
genomic_type_mapper = pickle.load(open(unprocessed_data_location + 'genomic_typing_dict.pkl', 'rb'))

***
**HGNC Data** 

_Human Gene Set Data_ - `hgnc_complete_set.txt`

In [ ]:
# download data
url = 'http://ftp.ebi.ac.uk/pub/databases/genenames/new/tsv/hgnc_complete_set.txt'
if not os.path.exists(unprocessed_data_location + 'hgnc_complete_set.txt'):
    data_downloader(url, unprocessed_data_location)

# load data
hgnc = pandas.read_csv(unprocessed_data_location + 'hgnc_complete_set.txt', header=0, delimiter='\t', low_memory=False)

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, updating data types (i.e. making all columns type `str`), and unnesting `|` delimited data. The final step is to update the gene_type variable such that each of the variable values is re-grouped to be protein-coding, other or ncRNA.

In [ ]:
hgnc = hgnc.loc[hgnc['status'].apply(lambda x: x == 'Approved')]
hgnc = hgnc[['hgnc_id', 'entrez_id', 'ensembl_gene_id', 'uniprot_ids', 'symbol', 'locus_type', 'alias_symbol', 'name', 'location', 'alias_name']]
hgnc.rename(columns={'uniprot_ids': 'uniprot_id', 'location': 'map_location', 'locus_type': 'hgnc_gene_type'}, inplace=True)
hgnc['hgnc_id'].replace('.*\:', '', inplace=True, regex=True)  # strip 'HGNC' off of the identifiers
hgnc.fillna('None', inplace=True)  # replace NaN with 'None'
hgnc['entrez_id'] = hgnc['entrez_id'].apply(lambda x: str(int(x)) if x != 'None' else 'None')  # make col str

# combine certain columns into single column
hgnc['name'] = hgnc['name'] + '|' + hgnc['alias_name']
hgnc['synonyms'] = hgnc['alias_symbol'] + '|' + hgnc['alias_name'] + '|' + hgnc['name']
hgnc['symbol'] = hgnc['symbol'] + '|' + hgnc['alias_symbol']

# explode nested data and reformat values in preparation for combining it with other gene identifiers
explode_df_hgnc = explodes_data(hgnc.copy(), ['ensembl_gene_id', 'uniprot_id', 'symbol', 'name', 'synonyms'], '|')

# reformat hgnc gene type
for val in genomic_type_mapper['hgnc_gene_type'].keys():
    explode_df_hgnc['hgnc_gene_type'].replace(val, genomic_type_mapper['hgnc_gene_type'][val], inplace=True)

# reformat master hgnc gene type
explode_df_hgnc['master_gene_type'] = explode_df_hgnc['hgnc_gene_type']
master_dict = genomic_type_mapper['hgnc_master_gene_type']
for val in master_dict.keys():
    explode_df_hgnc['master_gene_type'].replace(val, master_dict[val], inplace=True)

# post-process reformatted data
explode_df_hgnc.drop(['alias_symbol', 'alias_name'], axis=1, inplace=True)  # remove original gene type column
explode_df_hgnc.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_hgnc.head(n=3)

***
**Ensembl Data**

_Human Gene Set Data_ - `Homo_sapiens.GRCh38.102.gtf.gz`

In [ ]:
# download data
url = 'ftp://ftp.ensembl.org/pub/release-102/gtf/homo_sapiens/Homo_sapiens.GRCh38.102.gtf.gz'
if not os.path.exists(unprocessed_data_location + 'Homo_sapiens.GRCh38.102.gtf'):
    data_downloader(url, unprocessed_data_location)

# load data
ensembl_geneset = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.102.gtf',
                                  header = None, delimiter='\t', skiprows=5, usecols=[8], low_memory=False)

_Preprocess Data_  
Data file needs to be reformatted in order for it to be able to be merged with the other gene, RNA, and protein identifier data. To do this, we iterate over each row of the data and extract the fields shown below in `column_names`, making each of these extracted fields their own column. The final step is to update the gene_type variable such that each of the variable values is re-grouped to be `protein-coding`, `other` or `ncRNA`.

In [ ]:
ensembl_data = list(ensembl_geneset[8]); ensembl_df_data = []
for i in tqdm(range(0, len(ensembl_data))):
    if 'gene_id' in ensembl_data[i] and 'transcript_id' in ensembl_data[i]:
        row_dict = {x.split(' "')[0].lstrip(): x.split(' "')[1].strip('"') for x in ensembl_data[i].split(';')[0:-1]}
        ensembl_df_data += [(row_dict['gene_id'], row_dict['transcript_id'], row_dict['gene_name'],
                           row_dict['gene_biotype'], row_dict['transcript_name'], row_dict['transcript_biotype'])]
# convert to data frame
ensembl_geneset = pandas.DataFrame(ensembl_df_data,
                                   columns=['ensembl_gene_id', 'transcript_stable_id', 'symbol',
                                            'ensembl_gene_type', 'transcript_name', 'ensembl_transcript_type'])

# reformat ensembl gene type
gene_dict = genomic_type_mapper['ensembl_gene_type']
for val in gene_dict.keys(): ensembl_geneset['ensembl_gene_type'].replace(val, gene_dict[val], inplace=True)
# reformat master gene type
ensembl_geneset['master_gene_type'] = ensembl_geneset['ensembl_gene_type']
gene_dict = genomic_type_mapper['ensembl_master_gene_type']
for val in gene_dict.keys(): ensembl_geneset['master_gene_type'].replace(val, gene_dict[val], inplace=True)
# reformat master transcript type
ensembl_geneset['ensembl_transcript_type'].replace('vault_RNA', 'vaultRNA', inplace=True, regex=False)
ensembl_geneset['master_transcript_type'] = ensembl_geneset['ensembl_transcript_type']
trans_dict = genomic_type_mapper['ensembl_master_transcript_type']
for val in trans_dict.keys(): ensembl_geneset['master_transcript_type'].replace(val, trans_dict[val], inplace=True)

# post-process reformatted data
ensembl_geneset.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_geneset.head(n=3)

***
**Ensembl Annotation Data**

_Ensembl-UniProt_ - `Homo_sapiens.GRCh38.102.uniprot.tsv`  
Once the main ensembl gene set has been read in, the next step is to read in the `ensembl-uniprot` mapping file. These files are vital for successfully merging the ensembl identifiers with the uniprot data set.

In [ ]:
# download data
url_uniprot = 'ftp://ftp.ensembl.org/pub/release-102/tsv/homo_sapiens/Homo_sapiens.GRCh38.102.uniprot.tsv.gz'
if not os.path.exists(unprocessed_data_location + 'Homo_sapiens.GRCh38.102.uniprot.tsv'):
    data_downloader(url_uniprot, unprocessed_data_location)

# load data
ensembl_uniprot = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.102.uniprot.tsv', header=0, delimiter='\t', low_memory=False)

# preprocess data
ensembl_uniprot.rename(columns={'xref': 'uniprot_id', 'gene_stable_id': 'ensembl_gene_id'}, inplace=True)
ensembl_uniprot.replace('-', 'None', inplace=True)
ensembl_uniprot.fillna('None', inplace=True)
ensembl_uniprot = ensembl_uniprot.loc[ensembl_uniprot['xref_identity'].apply(lambda x: x != 'None')]
ensembl_uniprot = ensembl_uniprot.loc[ensembl_uniprot['uniprot_id'].apply(lambda x: '-' not in x)]  # remove isoforms
ensembl_uniprot = ensembl_uniprot.loc[ensembl_uniprot['info_type'].apply(lambda x: x == 'DIRECT')]
# ensembl_uniprot['master_gene_type'] = ['protein-coding'] * len(ensembl_uniprot)
# ensembl_uniprot['master_transcript_type'] = ['protein-coding'] * len(ensembl_uniprot)
ensembl_uniprot.drop(['db_name', 'info_type', 'source_identity', 'xref_identity', 'linkage_type'], axis=1, inplace=True)
ensembl_uniprot.drop_duplicates(subset=None, keep='first', inplace=True)

_Ensembl-Entrez_ - `Homo_sapiens.GRCh38.102.entrez.tsv`  
Once the main ensembl gene set has been read in, the next step is to read in the `ensembl-entrez` mapping file. These files are vital for successfully merging the ensembl identifiers with the entrez data set.

In [ ]:
# download data
url_entrez = 'ftp://ftp.ensembl.org/pub/release-102/tsv/homo_sapiens/Homo_sapiens.GRCh38.102.entrez.tsv.gz'
if not os.path.exists(unprocessed_data_location + 'Homo_sapiens.GRCh38.102.entrez.tsv'):
    data_downloader(url_entrez, unprocessed_data_location)

# load data
ensembl_entrez = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.GRCh38.102.entrez.tsv', header=0, delimiter='\t', low_memory=False)

# preprocess data
ensembl_entrez.rename(columns={'xref': 'entrez_id', 'gene_stable_id': 'ensembl_gene_id'}, inplace=True)
ensembl_entrez = ensembl_entrez.loc[ensembl_entrez['db_name'].apply(lambda x: x == 'EntrezGene')]
ensembl_entrez = ensembl_entrez.loc[ensembl_entrez['info_type'].apply(lambda x: x == 'DEPENDENT')]
ensembl_entrez.replace('-', 'None', inplace=True)
ensembl_entrez.fillna('None', inplace=True)
ensembl_entrez.drop(['db_name', 'info_type', 'source_identity', 'xref_identity', 'linkage_type'], axis=1, inplace=True)
ensembl_entrez.drop_duplicates(subset=None, keep='first', inplace=True)

_Merge Annotation Data_ - `ensembl_uniprot` + `ensembl_entrez`

In [ ]:
merge_cols = list(set(ensembl_entrez).intersection(set(ensembl_uniprot)))
ensembl_annot = pandas.merge(ensembl_uniprot, ensembl_entrez, on=merge_cols, how='outer')
ensembl_annot.fillna('None', inplace=True)

# preview data
ensembl_annot.head(n=3)

_Merge Ensembl Annotation and Gene Set Data_ - `ensembl_geneset` + `ensembl_annot`

In [ ]:
merge_cols = list(set(ensembl_annot).intersection(set(ensembl_geneset)))
ensembl = pandas.merge(ensembl_geneset, ensembl_annot, on=merge_cols, how='outer')
ensembl.fillna('None', inplace=True)
ensembl.replace('NA','None', inplace=True, regex=False)

# preview data
ensembl.head(n=3)

_Save Cleaned Ensembl Data_  
Save the cleaned Ensembl data so that it can be used when generating node metadata for transcript identifiers.

In [ ]:
ensembl.to_csv(processed_data_location + 'ensembl_identifier_data_cleaned.txt', header=True, sep='\t', index=False)

***
**UniProt Data**   
_Human Gene Set Data_ - `uniprot_identifier_mapping.tab`

This data was obtained by querying the [UniProt Knowledgebase](https://www.uniprot.org/uniprot/) using the *organism:"Homo sapiens (Human) [9606]"* keyword and including the following columns:
- Entry (Standard)    
- GeneID (*Genome Annotation*)  
- Ensembl (*Genome Annotation*)  
- HGNC (*Organism-specific*)  
- Gene names (primary) (*Names & Taxonomy*)    
- Gene synonym (primary) (*Names & Taxonomy*)    

The URL to access the results of this query is obtained by clicking on the share symbol and copying the free-text from the box. To obtain the data in a tab-delimited format the following string is appended to the end of the URL: "&format=tab".

**NOTE.** Be sure to obtain a new URL from the [UniProt Knowledgebase](https://www.uniprot.org/uniprot/) when rebuilding to ensure you are getting the most up-to-date data. This query was last generated on `12/02/2020`.

In [ ]:
# download data
url = 'https://www.uniprot.org/uniprot/?query=&fil=organism%3A%22Homo%20sapiens%20(Human)%20%5B9606%5D%22&columns=id%2Creviewed%2Cdatabase(GeneID)%2Cdatabase(Ensembl)%2Cdatabase(HGNC)%2Cgenes(ALTERNATIVE)%2Cgenes(PREFERRED)&format=tab'
if not os.path.exists(unprocessed_data_location + 'uniprot_identifier_mapping.tab'):
    data_downloader(url, unprocessed_data_location, 'uniprot_identifier_mapping.tab')

# load data
uniprot = pandas.read_csv(unprocessed_data_location + 'uniprot_identifier_mapping.tab', header=0, delimiter='\t')

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, and unnesting `"|"` delimited data.

In [ ]:
uniprot.fillna('None', inplace=True)  # replace NaN with 'None'
uniprot.rename(columns={'Entry': 'uniprot_id',
                        'Cross-reference (GeneID)': 'entrez_id',
                        'Ensembl transcript': 'transcript_stable_id',
                        'Cross-reference (HGNC)': 'hgnc_id',
                        'Gene names  (synonym )': 'synonyms',
                        'Gene names  (primary )' :'symbol'}, inplace=True)

# update space-delimited synonyms to a pipe (i.e. '|')
uniprot['synonyms'] = uniprot['synonyms'].apply(lambda x: '|'.join(x.split()) if x.isupper() else x)

# only keep reviewed entries
uniprot = uniprot.loc[uniprot['Status'].apply(lambda x: x != 'unreviewed')]

# explode nested data
explode_df_uniprot = explodes_data(uniprot.copy(), ['transcript_stable_id', 'entrez_id', 'hgnc_id'], ';')
explode_df_uniprot = explodes_data(explode_df_uniprot.copy(), ['symbol', 'synonyms'], '|')

# strip out uniprot names
explode_df_uniprot['transcript_stable_id'].replace('\s.*','', inplace=True, regex=True)

# remove duplicates
explode_df_uniprot.drop(['Status'], axis=1, inplace=True)
explode_df_uniprot.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_uniprot.head(n=3)

***
**NCBI Data**   
_Human Gene Set Data_ - `Homo_sapiens.gene_info`

In [ ]:
# download data
url = 'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz'
if not os.path.exists(unprocessed_data_location + 'Homo_sapiens.gene_info'):
    data_downloader(url, unprocessed_data_location)

# load data
ncbi_gene = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.gene_info', header=0, delimiter='\t', low_memory=False)

_Preprocess Data_  
Data file needs to be lightly cleaned before it can be merged with other data. This light cleaning includes renaming columns, replacing `NaN` with `None`, updating data types (i.e. making all columns type `str`), and unnesting `|` delimited data. Then, the `gene_type` variable is cleaned such that each of the variable's values are re-grouped to be `protein-coding`, `other` or `ncRNA`.

In [ ]:
# preprocess data
ncbi_gene = ncbi_gene.loc[ncbi_gene['#tax_id'].apply(lambda x: x == 9606)]  # remove non-human rows
ncbi_gene.replace('-', 'None', inplace=True)
ncbi_gene.rename(columns={'GeneID': 'entrez_id', 'Symbol': 'symbol', 'Synonyms': 'synonyms'}, inplace=True)
ncbi_gene['synonyms'] = ncbi_gene['synonyms'] + '|' + ncbi_gene['description'] + '|' + ncbi_gene['Full_name_from_nomenclature_authority'] + '|' + ncbi_gene['Other_designations']
ncbi_gene['symbol'] = ncbi_gene['Symbol_from_nomenclature_authority'] + '|' + ncbi_gene['symbol']
ncbi_gene['name'] = ncbi_gene['Full_name_from_nomenclature_authority'] + '|' + ncbi_gene['description']

# explode nested data
explode_df_ncbi_gene = explodes_data(ncbi_gene.copy(), ['symbol', 'synonyms', 'name', 'dbXrefs'], '|')

# clean up results
explode_df_ncbi_gene['entrez_id'] = explode_df_ncbi_gene['entrez_id'].astype(str)
explode_df_ncbi_gene = explode_df_ncbi_gene.loc[explode_df_ncbi_gene['dbXrefs'].apply(lambda x: x.split(':')[0] in ['Ensembl', 'HGNC', 'IMGT/GENE-DB'])]
explode_df_ncbi_gene['hgnc_id'] = explode_df_ncbi_gene['dbXrefs'].loc[explode_df_ncbi_gene['dbXrefs'].apply(lambda x: x.startswith('HGNC'))]
explode_df_ncbi_gene['ensembl_gene_id'] = explode_df_ncbi_gene['dbXrefs'].loc[explode_df_ncbi_gene['dbXrefs'].apply(lambda x: x.startswith('Ensembl'))]
explode_df_ncbi_gene.fillna('None', inplace=True)

# reformat entrez gene type
explode_df_ncbi_gene['entrez_gene_type'] = explode_df_ncbi_gene['type_of_gene']
gene_dict = genomic_type_mapper['entrez_gene_type']
for val in gene_dict.keys(): explode_df_ncbi_gene['entrez_gene_type'].replace(val, gene_dict[val], inplace=True)
# reformat master gene type
explode_df_ncbi_gene['master_gene_type'] = explode_df_ncbi_gene['entrez_gene_type']
gene_dict = genomic_type_mapper['master_gene_type']
for val in gene_dict.keys(): explode_df_ncbi_gene['master_gene_type'].replace(val, gene_dict[val], inplace=True)

# post-process reformatted data
explode_df_ncbi_gene.drop(['type_of_gene', 'dbXrefs', 'description', 'Nomenclature_status', 'Modification_date',
                           'LocusTag', '#tax_id', 'Full_name_from_nomenclature_authority', 'Feature_type',
                           'Symbol_from_nomenclature_authority'], axis=1, inplace=True)
explode_df_ncbi_gene['hgnc_id'] = explode_df_ncbi_gene['hgnc_id'].replace('HGNC:', '', regex=True)
explode_df_ncbi_gene['ensembl_gene_id'] = explode_df_ncbi_gene['ensembl_gene_id'].replace('Ensembl:', '', regex=True)
explode_df_ncbi_gene.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
explode_df_ncbi_gene.head(n=3)

***
**Protein Ontology Identifier Mapping Data**   
_Protein Ontology Identifier Data_ - `promapping.txt`

In [ ]:
# download data
url = 'https://proconsortium.org/download/current/promapping.txt'
if not os.path.exists(unprocessed_data_location + 'promapping.txt'):
    data_downloader(url, unprocessed_data_location)

# load data
pro_map = pandas.read_csv(unprocessed_data_location + 'promapping.txt', header=None, names=['pro_id', 'entry', 'pro_mapping'], delimiter='\t')

_Preprocess Data_  
Basic filtering to to include `Protein Ontology` mappings to `Uniprot` identifiers and cleaning to update formatting of accession values (i.e. removing `UniProtKB:`).

In [ ]:
pro_map = pro_map.loc[pro_map['entry'].apply(lambda x: x.startswith('Uni') and '_VAR' not in x and ', ' not in x)]  # keep 'UniProtKB' rows
pro_map = pro_map.loc[pro_map['pro_mapping'].apply(lambda x: x.startswith('exact'))] # keep exact mappings
pro_map['pro_id'].replace('PR:','PR_', inplace=True, regex=True)  # replace PR: with PR_
pro_map['entry'].replace('(^\w*\:)','', inplace=True, regex=True)  # remove id prefixes
pro_map = pro_map.loc[pro_map['pro_id'].apply(lambda x: '-' not in x)] # remove isoforms
pro_map.rename(columns={'entry': 'uniprot_id'}, inplace=True)  # rename columns before merging
pro_map.drop(['pro_mapping'], axis=1, inplace=True)  # remove uneeded columns
pro_map.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
pro_map.head(n=3)

***

#### Merging Processed Genomic Identifier Data Sources  
Merging all of the genomic identifier data sources is needed in order to create a map that can be used to integrate the different genomic data sources.

***
*Data Sources:* `hgnc` + `ensembl`

In [ ]:
merge_cols = list(set(explode_df_hgnc.columns).intersection(set(ensembl.columns)))
ensembl_hgnc_merged_data = pandas.merge(ensembl, explode_df_hgnc, on=merge_cols, how='outer')

# clean up merged data
ensembl_hgnc_merged_data.fillna('None', inplace=True)
ensembl_hgnc_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_merged_data.head(n=3)

***
*Data Sources:* `ensembl_hgnc_merged_data` + `explode_df_uniprot`

In [ ]:
merge_cols = list(set(ensembl_hgnc_merged_data.columns).intersection(set(explode_df_uniprot.columns)))
ensembl_hgnc_uniprot_merged_data = pandas.merge(ensembl_hgnc_merged_data, explode_df_uniprot, on=merge_cols, how='outer')

# clean up merged data
ensembl_hgnc_uniprot_merged_data.fillna('None', inplace=True)
ensembl_hgnc_uniprot_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_uniprot_merged_data.head(n=3)

***
*Data Sources:* `ensembl_hgnc_uniprot_merged_data` + `Homo_sapiens.gene_info`

In [ ]:
merge_cols = merge_cols = list(set(ensembl_hgnc_uniprot_merged_data).intersection(set(explode_df_ncbi_gene.columns)))
ensembl_hgnc_uniprot_ncbi_merged_data = pandas.merge(ensembl_hgnc_uniprot_merged_data, explode_df_ncbi_gene, on=merge_cols, how='outer')

# clean up merged data
ensembl_hgnc_uniprot_ncbi_merged_data.fillna('None', inplace=True)
ensembl_hgnc_uniprot_ncbi_merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
ensembl_hgnc_uniprot_ncbi_merged_data.head(n=3)

***
*Data Sources:* `ensembl_hgnc_uniprot_ncbi_merged_data` + `promapping.txt`  

In [ ]:
merged_data = pandas.merge(ensembl_hgnc_uniprot_ncbi_merged_data, pro_map, on='uniprot_id', how='outer')

# clean up merged data
merged_data.fillna('None', inplace=True)
merged_data.drop_duplicates(subset=None, keep='first', inplace=True)

# preview data
merged_data.head(n=3)

_Fix Symbol Formatting_  
some genes are formatted similarly to dates (e.g. `DEC1`), which can be erroneously re-formatted during input as a date value (i.e. `1-DEC`). In order for the data to be successfully merged with other data sources, all date-formatted genes need to be resolved.

In [ ]:
clean_dates = []
for x in tqdm(list(merged_data['symbol'])):
    if '-' in x and len(x.split('-')[0]) < 3 and len(x.split('-')[1]) == 3:
        clean_dates.append(x.split('-')[1].upper() + x.split('-')[0])
    else: clean_dates.append(x)

# add cleaned date var back to data set
merged_data['symbol'] = clean_dates
merged_data.fillna('None', inplace=True)

# make sure that all gene and transcript type colunmns have none recoded to unknown or not protein-coding
merged_data['hgnc_gene_type'].replace('None', 'unknown', inplace=True, regex=False)
merged_data['ensembl_gene_type'].replace('None', 'unknown', inplace=True, regex=False)
merged_data['entrez_gene_type'].replace('None', 'unknown', inplace=True, regex=False)
merged_data['master_gene_type'].replace('None', 'unknown', inplace=True, regex=False)
merged_data['master_transcript_type'].replace('None', 'not protein-coding', inplace=True, regex=False)
merged_data['ensembl_transcript_type'].replace('None', 'unknown', inplace=True, regex=False)

# remove duplicates
merged_data_clean = merged_data.drop_duplicates(subset=None, keep='first')

# write data
merged_data_clean.to_csv(processed_data_location + 'Merged_Human_Ensembl_Entrez_HGNC_Uniprot_Identifiers.txt', header=True, sep='\t', index=False)
    
# preview data
merged_data_clean.head(n=3)

***
**Create a Master Mapping Dictionary**  
Although the above steps result in a `pandas.Dataframe` of the merged identifiers, there is still work needed in order to be able to obtain a complete mapping between the identifiers. For example, if you were to search for Entrez gene identifier `entrez_259234` you would find the following mappings: `entrez_259234-ENSG00000233316`, `entrez_259234-DSCR10`. If you only had `ENSG00000233316`, with the current data you would be unable to obtain the gene symbol without first mapping to the Entrez gene identifier. 

To solve this problem, we build a master dictionary where the keys are `ensembl_gene_id`, `transcript_stable_id`, `protein_stable_id`, `uniprot_id`, `entrez_id`, `hgnc_id`, `pro_id`, and `symbol` identifiers and values are the list of genomic identifiers that match to each identifier. It's important to note that there are several labeling identifiers (i.e. `name`, `chromosome`, `map_location`, `Other_designations`, `synonyms`, `transcript_name`, `*_gene_types`, and `trasnscript_type_update`), which will only be mapped when clustered against one of the primary identifier types (i.e. the keys described above).

_Note_. The next chunk does a lot of heavy lifting and takes approximately ~40 minutes to run.

In [ ]:
# reformat data to convert all nones, empty values, and unknowns to NaN
for col in merged_data_clean.columns:
    merged_data_clean[col] = merged_data_clean[col].apply(lambda x: '|'.join([i for i in x.split('|') if i != 'None']))
merged_data_clean.replace(to_replace=['None', '', 'unknown'], value=numpy.nan, inplace=True)
identifiers = [x for x in merged_data_clean.columns if x.endswith('_id')] + ['symbol']

In [ ]:
# convert data to dictionary
master_dict = {}
for idx in tqdm(identifiers):
    grouped_data = merged_data_clean.groupby(idx)
    grp_ids = set([x for x in list(grouped_data.groups.keys()) if x != numpy.nan])
    for grp in grp_ids:
        df = grouped_data.get_group(grp).dropna(axis=1, how='all')
        df_cols, key = df.columns, idx + '_' + grp
        val_df = [[col + '_' + x for x in set(df[col]) if isinstance(x, str)] for col in df_cols if col != idx]
        if len(val_df) > 0:
            if key in master_dict.keys(): master_dict[key] += [i for j in val_df for i in j if len(i) > 0]
            else: master_dict[key] = [i for j in val_df for i in j if len(i) > 0]  

_Finalizing Master Mapping Dictionary_  
Then, we need to identify a master gene and transcript type for each entity because the last ran code chunk can result in several genes and transcripts with differing types (i.e. `protein-coding` or `not protein-coding`). The next step collects all information for each gene and transcript and performs a voting procedure to select a single primary gene and transcript type.

In [ ]:
reformatted_mapped_identifiers = dict()
for key, values in tqdm(master_dict.items()):
    identifier_info = set(values); gene_prefix = 'master_gene_type_'; trans_prefix = 'master_transcript_type_'
    if key.split('_')[0] in ['protein', 'uniprot', 'pro']: pass
    elif 'transcript' in key:
        trans_match = [x.replace(trans_prefix, '') for x in values if trans_prefix in x]
        if len(trans_match) > 0:
            t_type_list = ['protein-coding' if ('protein-coding' in trans_match or 'protein_coding' in trans_match) else 'not protein-coding']
            identifier_info |= {'transcript_type_update_' + max(set(t_type_list), key=t_type_list.count)}
    else:
        gene_match = [x.replace(gene_prefix, '') for x in values if x.startswith(gene_prefix) and 'type' in x]
        if len(gene_match) > 0:
            g_type_list = ['protein-coding' if ('protein-coding' in gene_match or 'protein_coding' in gene_match) else 'not protein-coding']
            identifier_info |= {'gene_type_update_' + max(set(g_type_list), key=g_type_list.count)}
    reformatted_mapped_identifiers[key] = identifier_info

In [ ]:
# save a copy of the dictionary
# output > 4GB requires special approach: https://stackoverflow.com/questions/42653386/does-pickle-randomly-fail-with-oserror-on-large-files
filepath = processed_data_location + 'Merged_gene_rna_protein_identifiers.pkl'

# defensive way to write pickle.write, allowing for very large files on all platforms
max_bytes, bytes_out = 2**31 - 1, pickle.dumps(reformatted_mapped_identifiers)
n_bytes = sys.getsizeof(bytes_out)

with open(filepath, 'wb') as f_out:
    for idx in range(0, n_bytes, max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])

In [ ]:
# # load data
# filepath = processed_data_location + 'Merged_gene_rna_protein_identifiers.pkl'

# # defensive way to write pickle.load, allowing for very large files on all platforms
# max_bytes = 2**31 - 1
# input_size = os.path.getsize(filepath)
# bytes_in = bytearray(0)

# with open(filepath, 'rb') as f_in:
#     for _ in range(0, input_size, max_bytes):
#         bytes_in += f_in.read(max_bytes)

# # load ickled data
# reformatted_mapped_identifiers = pickle.loads(bytes_in)

***
### Ensembl Gene-Entrez Gene <a class="anchor" id="ensemblgene-entrezgene"></a>


**Purpose:** To map Ensembl gene identifiers to Entrez gene identifiers when creating `gene`-`gene` edges

**Output:** `ENSEMBL_GENE_ENTREZ_GENE_MAP.txt`

In [ ]:
genomic_id_mapper(reformatted_mapped_identifiers,
                  processed_data_location + 'ENSEMBL_GENE_ENTREZ_GENE_MAP.txt',
                  'ensembl_gene_id', 'entrez_id', 'ensembl_gene_type', 'entrez_gene_type',
                  'gene_type_update', 'gene_type_update')

In [ ]:
# load data, print the number of rows, and preview it
egeg_data = pandas.read_csv(processed_data_location + 'ENSEMBL_GENE_ENTREZ_GENE_MAP.txt',
                            header=None, delimiter='\t', low_memory=False,
                            names=['Ensembl_Gene_IDs', 'Entrez_Gene_IDs',
                                   'Ensembl_Gene_Type', 'Entrez_Gene_Type',
                                   'Master_Gene_Type1', 'Master_Gene_Type2'])

print('There are {edge_count} ensembl gene-entrez gene edges'.format(edge_count=len(egeg_data)))
egeg_data.head(n=5)

***
### Ensembl Transcript-Protein Ontology <a class="anchor" id="ensembltranscript-proteinontology"></a>

**Purpose:** To map Ensembl transcript identifiers to Protein Ontology identifiers when creating `rna`-`protein` edges

**Output:** `ENSEMBL_TRANSCRIPT_PROTEIN_ONTOLOGY_MAP.txt`


In [ ]:
genomic_id_mapper(reformatted_mapped_identifiers,
                  processed_data_location + 'ENSEMBL_TRANSCRIPT_PROTEIN_ONTOLOGY_MAP.txt',
                  'transcript_stable_id', 'pro_id', 'ensembl_transcript_type', None,
                  'transcript_type_update', None)

In [ ]:
# load data, print the number of rows, and preview it
etpr_data = pandas.read_csv(processed_data_location + 'ENSEMBL_TRANSCRIPT_PROTEIN_ONTOLOGY_MAP.txt',
                            header=None, delimiter='\t', low_memory=False, usecols=[0, 1, 2, 4],
                            names=['Ensembl_Transcript_IDs', 'Protein_Ontology_IDs',
                                   'Ensembl_Transcript_Type', 'Master_Transcript_Type'])

print('There are {edge_count} ensembl transcript-protein ontology edges'.format(edge_count=len(etpr_data)))
etpr_data.head(n=5)

***
### Entrez Gene-Ensembl Transcript <a class="anchor" id="entrezgene-ensembltranscript"></a>

**Purpose:** To map entrez gene identifiers to Ensembl transcript identifiers when creating `gene`-`rna` edges

**Output:** `ENTREZ_GENE_ENSEMBL_TRANSCRIPT_MAP.txt`

In [ ]:
genomic_id_mapper(reformatted_mapped_identifiers,
                  processed_data_location + 'ENTREZ_GENE_ENSEMBL_TRANSCRIPT_MAP.txt',
                  'entrez_id', 'transcript_stable_id', 'entrez_gene_type', 'ensembl_transcript_type',
                  'gene_type_update', 'transcript_type_update')

In [ ]:
# load data, print the number of rows, and preview it
eet_data = pandas.read_csv(processed_data_location + 'ENTREZ_GENE_ENSEMBL_TRANSCRIPT_MAP.txt',
                           header=None, delimiter='\t', low_memory=False,
                           names=['Entrez_Gene_IDs', 'Ensembl_Transcript_IDs',
                                  'Entrez_Gene_Type', 'Ensembl_Transcript_Type',
                                  'Master_Gene_Type', 'Master_Transcript_Type'])

print('There are {edge_count} entrez gene identifiers-ensembl transcript edges'.format(edge_count=len(eet_data)))
eet_data.head(n=5)

***
### Entrez Gene-Protein Ontology <a class="anchor" id="entrezgene-proteinontology"></a>

**Purpose:** To map Protein Ontology identifiers to Ensembl transcript identifiers when creating the following edges:   
- chemical-protein  
- gene-protein

**Output:** `ENTREZ_GENE_PRO_ONTOLOGY_MAP.txt`

In [ ]:
genomic_id_mapper(reformatted_mapped_identifiers,
                  processed_data_location + 'ENTREZ_GENE_PRO_ONTOLOGY_MAP.txt',
                  'entrez_id', 'pro_id', 'entrez_gene_type', None,
                  'gene_type_update', None)

In [ ]:
# load data, print the number of rows, and preview it
egpr_data = pandas.read_csv(processed_data_location + 'ENTREZ_GENE_PRO_ONTOLOGY_MAP.txt',
                            header=None, delimiter='\t', low_memory=False, usecols = [0, 1, 2, 4],
                            names=['Gene_IDs', 'Protein_Ontology_IDs',
                                   'Entrez_Gene_Type', 'Master_Gene_Type'])

print('There are {edge_count} entrez gene-protein ontology edges'.format(edge_count=len(egpr_data)))
egpr_data.head(n=5)

***
### Gene Symbol-Ensembl Transcript <a class="anchor" id="genesymbol-ensembltranscript"></a>

**Purpose:** To map gene symbols to Ensembl transcript identifiers when creating the following edges: 
- chemical-rna  
- rna-anatomy  
- rna-cell  

**Output:** `GENE_SYMBOL_ENSEMBL_TRANSCRIPT_MAP.txt`

In [ ]:
genomic_id_mapper(reformatted_mapped_identifiers,
                  processed_data_location + 'GENE_SYMBOL_ENSEMBL_TRANSCRIPT_MAP.txt',
                  'symbol', 'transcript_stable_id', 'master_gene_type', 'ensembl_transcript_type',
                  'gene_type_update', 'transcript_type_update')

In [ ]:
# load data, print the number of rows, and preview it
set_data = pandas.read_csv(processed_data_location + 'GENE_SYMBOL_ENSEMBL_TRANSCRIPT_MAP.txt',
                           header=None, delimiter='\t', low_memory=False,
                           names=['Gene_Symbols', 'Ensembl_Transcript_IDs',
                                  'Gene_Type', 'Ensembl_Transcript_Type',
                                  'Master_Gene_Type', 'Master_Transcript_Type'])

print('There are {edge_count} gene symbol-ensembl transcript edges'.format(edge_count=len(set_data.drop_duplicates())))
set_data.head(n=5)

***

### STRING-Protein Ontology <a class="anchor" id="string-proteinontology"></a>

**Purpose:** To map STRING identifiers to Protein Ontology identifiers when creating `protein`-`protein` edges 

**Output:** `STRING_PRO_ONTOLOGY_MAP.txt`

In [ ]:
genomic_id_mapper(reformatted_mapped_identifiers,
                  processed_data_location + 'STRING_PRO_ONTOLOGY_MAP.txt',
                  'protein_stable_id', 'pro_id', None, None, None, None)

In [ ]:
# load data, print the number of rows, and preview it
stpr_data = pandas.read_csv(processed_data_location + 'STRING_PRO_ONTOLOGY_MAP.txt',
                            header=None, delimiter='\t', low_memory=False, usecols=[0, 1],
                            names=['STRING_IDs', 'Protein_Ontology_IDs'])

print('There are {edge_count} string-protein ontology edges'.format(edge_count=len(stpr_data.drop_duplicates())))
stpr_data.head(n=5)

***

### Uniprot Accession-Protein Ontology <a class="anchor" id="uniprotaccession-proteinontology"></a>

**Purpose:** To map Uniprot accession identifiers to Protein Ontology identifiers when creating the following edges:  
- protein-gobp  
- protein-gomf  
- protein-gocc  
- protein-cofactor  
- protein-catalyst 
- protein-pathway

**Output:** `UNIPROT_ACCESSION_PRO_ONTOLOGY_MAP.txt`

In [ ]:
genomic_id_mapper(reformatted_mapped_identifiers,
                  processed_data_location + 'UNIPROT_ACCESSION_PRO_ONTOLOGY_MAP.txt',
                  'uniprot_id', 'pro_id', None, None, None, None)

In [ ]:
# load data, print the number of rows, and preview it
uapr_data = pandas.read_csv(processed_data_location + 'UNIPROT_ACCESSION_PRO_ONTOLOGY_MAP.txt',
                            header=None, delimiter='\t', low_memory=False, usecols=[0, 1],
                            names=['Uniprot_Accession_IDs', 'Protein_Ontology_IDs'])

print('There are {edge_count} uniprot accession-protein ontology edges'.format(edge_count=len(uapr_data.drop_duplicates())))
uapr_data.head(n=5)

<br>

***
***
### Other Identifier Mapping <a class="anchor" id="other-identifier-mapping"></a>
***
* [ChEBI Identifiers](#mesh-chebi)  
* [Human Protein Atlas Tissue and Cell Types](#hpa-uberon) 
* [Human Disease and Phenotype Identifiers](#disease-identifiers) 
* [Reactome Pathways and the Pathway Ontology](#reactome-pw)  
* [Genomic Identifiers and the Sequence Ontology](#genomic-so)  

***
***

### ChEBI-MeSH Identifiers <a class="anchor" id="mesh-chebi"></a>

**Data Source Wiki Page:** [mapping-mesh-to-chebi](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#mapping-mesh-identifiers-to-chebi-identifiers)  

**Purpose:** Map MeSH identifiers to ChEBI identifiers when creating the following edges:  
- chemical-gene  
- chemical-disease

**Dependencies:** Recapitulates the [`LOOM`](https://www.bioontology.org/wiki/BioPortal_Mappings) algorithm implemented by BioPortal when creating mappings between resources. The procedure is relatively straightforward and consists of the following:
- For all MeSH `SCR Chemicals`, obtain the following information:  
  - <u>Identifiers</u>: MeSH identifiers     
  - <u>Labels</u>: string labels using the `RDFS:label` object property  
  - <u>Synonyms</u>: track down all synonyms using the `vocab:concept` and `vocab:preferredConcept` object properties   
- For all ChEBI classes, obtain the following information:  
  - <u>Labels</u>: string labels using the `RDFS:label` object property  
  - <u>Synonyms</u>: track down all synonyms using all `synonym` object properties 
  
*Alternatively:* You can use the [`ncbo_rest_api.py`](https://gist.github.com/callahantiff/a28fb3160782f42f104e9ec41553af0d) script to pull mappings from the BioPortal API, but note that it takes >2 days for it to finish.

**Output:** `CHEBI_MESH_MAP.txt`


***  
**MeSH**  
Downloads the `nt`-formatted version of the current MeSH vocabulary. Preprocesing is then performed in order to reformat the data so that it can be converted into a Pandas DataFrame in preparation of merging it with `ChEBI` in order to identify overlapping concepts.

In [ ]:
# download data
url = 'ftp://nlmpubs.nlm.nih.gov/online/mesh/rdf/2021/mesh2021.nt'
if not os.path.exists(unprocessed_data_location + 'mesh2021.nt'):
    data_downloader(url, unprocessed_data_location)
    
# load data
mesh = [x.split('> ') for x in tqdm(open(unprocessed_data_location + 'mesh2021.nt').readlines())]

In [ ]:
# preprocess data
mesh_dict, results = {}, []
for row in tqdm(mesh):
    dbx, lab, msh_type = None, None, None
    s, p, o = row[0].split('/')[-1], row[1].split('#')[-1], row[2]  
    if s[0] in ['C', 'D'] and ('.' not in s and 'Q' not in s) and len(s) >= 5:
        s = 'MESH_' + s
        if p == 'preferredConcept' or p == 'concept': dbx = 'MESH_' + o.split('/')[-1]
        if 'label' in p.lower(): lab = o.split('"')[1]
        if 'type' in p.lower(): msh_type = o.split('#')[1]
        if s in mesh_dict.keys():
            if dbx is not None: mesh_dict[s]['dbxref'].add(dbx)
            if lab is not None: mesh_dict[s]['label'].add(lab)
            if msh_type is not None: mesh_dict[s]['type'].add(msh_type)
        else:
            mesh_dict[s] = {'dbxref': set() if dbx is None else {dbx},
                            'label': set() if lab is None else {lab},
                            'type': set() if msh_type is None else {msh_type},
                            'synonym': set()}

# fine tune dictionary - obtain labels for each entry's synonym identifiers
for key in tqdm(mesh_dict.keys()):
    for i in mesh_dict[key]['dbxref']:
        if len(mesh_dict[key]['dbxref']) > 0 and i in mesh_dict.keys():
            mesh_dict[key]['synonym'] |= mesh_dict[i]['label']

# expand data and convert to pandas DataFrame
for key, value in tqdm(mesh_dict.items()):
    results += [[key, list(value['label'])[0], 'NAME']]
    if len(value['synonym']) > 0:
        for i in value['synonym']:
            results += [[key, i, 'SYNONYM']]
mesh_filtered = pandas.DataFrame({'CODE': [x[0] for x in results],
                                  'TYPE': [x[2] for x in results],
                                  'STRING': [x[1] for x in results]})

# lowercase all strings and remove white space and punctuation
mesh_filtered['STRING'] = mesh_filtered['STRING'].str.lower()
mesh_filtered['STRING'] = mesh_filtered['STRING'].str.replace('[^\w]','')

# preview data
mesh_filtered.head()

***  
**ChEBI**  
Downloads the flat-file containing labels and synonyms for all classes in the `ChEBI` ontology. Preprocessing is then performed in order to reformat the data so that it can be converted into a Pandas DataFrame in preparation of merging it with `MeSH` in order to identify overlapping concepts.

In [ ]:
# download data
url = 'ftp://ftp.ebi.ac.uk/pub/databases/chebi/Flat_file_tab_delimited/names.tsv.gz'
if not os.path.exists(unprocessed_data_location + 'names.tsv'):
    data_downloader(url, unprocessed_data_location)
    
# load data
chebi = pandas.read_csv(unprocessed_data_location + 'names.tsv', header=0, delimiter='\t')

# preprocess data
chebi_filtered = chebi[['COMPOUND_ID', 'TYPE', 'NAME']]
chebi_filtered.drop_duplicates(subset=None, keep='first', inplace=True)
chebi_filtered.columns = ['CODE', 'TYPE', 'STRING']

# append CHEBI to the number in each code
chebi_filtered['CODE'] = chebi_filtered['CODE'].apply(lambda x: "{}{}".format('CHEBI_', x))

# lowercase all strings and remove white space and punctuation
chebi_filtered['STRING'] = chebi_filtered['STRING'].str.lower()
chebi_filtered['STRING'] = chebi_filtered['STRING'].str.replace('[^\w]','')

# preview data
chebi_filtered.head()

***  
**Merge Identifier Data**  
Performs an inner merge of the `MeSH` and `ChEBI` Pandas DataFrames in order to find concepts that exist in both DataFrames. Results are then written out to a text file.

In [ ]:
# merge data
chem_merge = pandas.merge(chebi_filtered[['STRING', 'CODE']], mesh_filtered[['STRING', 'CODE']], on='STRING', how='inner')

# filter results
mesh_edges = set()
for idx, row in chem_merge.drop_duplicates().iterrows():
    mesh, chebi = row['CODE_y'], row['CODE_x']
    syns = [x for x in mesh_dict[mesh]['dbxref'] if 'C' in x or 'D' in x]
    mesh_edges.add(tuple([mesh, chebi]))
    if len(syns) > 0:
        for x in syns:
            mesh_edges.add(tuple([x, chebi]))

# write resulting mappings
with open(processed_data_location + 'MESH_CHEBI_MAP.txt', 'w') as out:
    for pair in mesh_edges:
        out.write(pair[0] + '\t' + pair[1] + '\n')

In [ ]:
# load data
data = pandas.read_csv(processed_data_location + 'MESH_CHEBI_MAP.txt', header=None, names=['MESH_ID', 'CHEBI_ID'], delimiter='\t')

# preview mapping results
print('There are {} MeSH-ChEBI Edges'.format(len(data)))
data.head(n=5)

***

### Disease and Phenotype Identifiers <a class="anchor" id="disease-identifiers"></a>

**Data Source Wiki Page:** [DisGeNET](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#disgenet)  

**Purpose:** This script downloads the Human Phenotype Ontology (HPO), the MonDO Disease Ontology (MONDO), and [disease_mappings.tsv](https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_mappings.tsv.gz) in order to map UMLS identifiers to HPO and MONDO identifiers when creating the following edges:  
- chemical-disease  
- disease-phenotype  
- chemical-phenotype  
- gene-phenotype  
- variant-phenotype  

**Output:**   
- Human Disease Ontology Mappings ➞ `DISEASE_MONDO_MAP.txt`
- Human Phenotype Ontology Mappings ➞ `PHENOTYPE_HPO_MAP.txt`

***
**MONDO Identifiers**  
`MONDO` contains DbXRef mappings to other disease terminology identifiers. To make this useful, we will store the DbXRefs as a dictionary with `MONDO` identifiers as the values.

In [ ]:
# download ontology
if not os.path.exists(unprocessed_data_location + 'mondo_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/mondo.owl',
                             unprocessed_data_location + 'mondo_with_imports.owl'))
    
# read data into RDFLib graph object
mondo_graph = Graph().parse(unprocessed_data_location + 'mondo_with_imports.owl')
print('There are {} axioms in the ontology (date: {})'.format(len(mondo_graph), datetime.datetime.now().strftime('%m/%d/%Y')))

# get dbxrefs for all MONDO classes
dbxref_res = gets_ontology_class_dbxrefs(mondo_graph)[0]
mondo_dict = {str(k).lower().split('/')[-1]: {str(i).split('/')[-1].replace('_', ':') for i in v} for k, v in dbxref_res.items() if 'MONDO' in str(v)}

# pickle dictionary
pickle.dump(mondo_dict, open(processed_data_location + 'Mondo_Identifier_Map.pkl', 'wb'), protocol=4)

***
**HPO Identifiers**  
`HPO` contains DbXRef mappings to other disease terminology identifiers. To make this useful, we will store the DbXRefs as a dictionary with `HPO` identifiers as the values.

In [ ]:
# download ontology
if not os.path.exists(unprocessed_data_location + 'hp_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/hp.owl',
                             unprocessed_data_location + 'hp_with_imports.owl'))

# read data into RDFLib graph object
hp_graph = Graph().parse(unprocessed_data_location + 'hp_with_imports.owl')
print('There are {} axioms in the ontology (date: {})'.format(len(hp_graph), datetime.datetime.now().strftime('%m/%d/%Y')))

# get dbxrefs for all HPO classes
dbxref_res = gets_ontology_class_dbxrefs(hp_graph)[0]
hp_dict = {str(k).lower().split('/')[-1]: {str(i).split('/')[-1].replace('_', ':') for i in v} for k, v in dbxref_res.items() if 'HP' in str(v)}

# pickle dictionary
pickle.dump(hp_dict, open(processed_data_location + 'HPO_Identifier_Map.pkl', 'wb'), protocol=4)

***
**DisGeNET Disease Mappings**

In [ ]:
# download data
url = 'https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_mappings.tsv.gz'
if not os.path.exists(unprocessed_data_location + 'disease_mappings.tsv'):
    data_downloader(url, unprocessed_data_location)
    
# load data
disease_data = pandas.read_csv(unprocessed_data_location + 'disease_mappings.tsv', header=0, delimiter='\t')

# reformat data
disease_data['vocabulary'] = disease_data['vocabulary'].str.lower()
disease_data['diseaseId'] = disease_data['diseaseId'].str.lower()
disease_data['vocabulary'] = ['doid' if x == 'do' else 'ordoid' if x == 'ordo' else x for x in disease_data['vocabulary']]

# preview data
disease_data.head(n=3)

_Build Disease Identifier Dictionary_  
In order to improve efficiency when mapping different disease terminology identifiers to the [MonDO Disease Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#mondo-disease-ontology) and [Human Phenotype Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#human-phenotype-ontology), we create a dictionary of disease identifiers.

In [ ]:
# get all CUIs found with HPO and MONDO
disease_data_keep = disease_data.query('vocabulary == "hpo" | vocabulary == "mondo"')

# create mondo and hpo dictionary
hp_mondo_dict = {}
for idx, row in tqdm(disease_data_keep.iterrows(), total=disease_data_keep.shape[0]):
    if row['vocabulary'] == 'mondo': key, value = 'umls:' + row['diseaseId'], 'MONDO:' + row['code']
    else: key, value = 'umls:' + row['diseaseId'], row['code']
    if key in hp_mondo_dict.keys(): hp_mondo_dict[key] |= {value}
    else: hp_mondo_dict[key] = {value}
# add ontology mappings from MONDO and HPO
for key in tqdm(hp_mondo_dict.keys()):
    if key in mondo_dict.keys():
        hp_mondo_dict[key] = set(list(hp_mondo_dict[key]) + list(mondo_dict[key]))
    if key in hp_dict.keys():
        hp_mondo_dict[key] = set(list(hp_mondo_dict[key]) + list(hp_dict[key]))

In [ ]:
# get all rows for HPO/MONDO CUIs to obtain mappings to other disease identifiers
disease_data_other = disease_data[disease_data.diseaseId.isin(disease_data_keep['diseaseId'])]

# get all other codes that map to MONDO or HPO by hopping through MONDO/HPO relevant CUIs
disease_dict = {}
for idx, row in tqdm(disease_data_other.iterrows(), total=disease_data_other.shape[0]):
    if row['vocabulary'] == 'mondo' or row['vocabulary'] == 'hpo':
        key, value = 'umls:' + row['diseaseId'].lower(), row['code']
        if key in disease_dict.keys(): disease_dict[key] |= {value}
        else: disease_dict[key] = {value}
    else:
        if 'mondo' not in row['code'] or 'hp' not in row['code']:
            if ':' not in row['code']: key, value = row['vocabulary'] + ':' + row['code'], hp_mondo_dict['umls:' + row['diseaseId']]
            else: key, value = row['code'], hp_mondo_dict['umls:' + row['diseaseId']]
            if key in disease_dict.keys(): disease_dict[key] |= value
            else: disease_dict[key] = value

# add ontology dictionaries
disease_dict = {**disease_dict, **mondo_dict, **hp_dict}

_Write Mapping Data_

In [ ]:
with open(processed_data_location + 'DISEASE_MONDO_MAP.txt', 'w') as outfile1, open(processed_data_location + 'PHENOTYPE_HPO_MAP.txt', 'w') as outfile2:
    for k, v in tqdm(disease_dict.items()):
        if any(x for x in v if x.startswith('MONDO')):
            for idx in [x.replace(':', '_') for x in v if 'MONDO' in x]:
                outfile1.write(k.upper().split(':')[-1] + '\t' + idx + '\n')
        if any(x for x in v if x.startswith('HP')):
            for idx in [x.replace(':', '_') for x in v if 'HP' in x]:
                outfile2.write(k.upper().split(':')[-1]  + '\t' + idx + '\n')

_Preview Processed MONDO Disease Ontology Mappings_

In [ ]:
# load data, print row count, and preview it
dis_data = pandas.read_csv(processed_data_location + 'DISEASE_MONDO_MAP.txt', header=None, names=['Disease_IDs', 'MONDO_IDs'], delimiter='\t')

print('There are {} disease-MONDO edges'.format(len(dis_data)))
dis_data.head(n=5)

_Preview Processed Human Phenotype Mappings_

In [ ]:
# load data, print row count, and preview it
hp_data = pandas.read_csv(processed_data_location + 'PHENOTYPE_HPO_MAP.txt', header=None, names=['Disease_IDs', 'HP_IDs'], delimiter='\t')

print('There are {} phenotype-HPO edges'.format(len(hp_data)))
hp_data.head(n=5)

***

### Human Protein Atlas/GTEx Tissue/Cells - UBERON + Cell Ontology + Cell Line Ontology <a class="anchor" id="hpa-uberon"></a>

**Data Source Wiki Page:**  
- [human-protein-atlas](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#human-protein-atlas) 
- [genotype-tissue-expression-project](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#the-genotype-tissue-expression-gtex-project)  

<br>

**Purpose:** Downloads a query for cell, tissue, and blood types with overexpressed protein-coding genes in the human proteome ([`proteinatlas_search.tsv`](https://www.proteinatlas.org/api/search_download.php?search=&columns=g,eg,up,pe,rnatsm,rnaclsm,rnacasm,rnabrsm,rnabcsm,rnablsm,scl,t_RNA_adipose_tissue,t_RNA_adrenal_gland,t_RNA_amygdala,t_RNA_appendix,t_RNA_basal_ganglia,t_RNA_bone_marrow,t_RNA_breast,t_RNA_cerebellum,t_RNA_cerebral_cortex,t_RNA_cervix,_uterine,t_RNA_colon,t_RNA_corpus_callosum,t_RNA_ductus_deferens,t_RNA_duodenum,t_RNA_endometrium_1,t_RNA_epididymis,t_RNA_esophagus,t_RNA_fallopian_tube,t_RNA_gallbladder,t_RNA_heart_muscle,t_RNA_hippocampal_formation,t_RNA_hypothalamus,t_RNA_kidney,t_RNA_liver,t_RNA_lung,t_RNA_lymph_node,t_RNA_midbrain,t_RNA_olfactory_region,t_RNA_ovary,t_RNA_pancreas,t_RNA_parathyroid_gland,t_RNA_pituitary_gland,t_RNA_placenta,t_RNA_pons_and_medulla,t_RNA_prostate,t_RNA_rectum,t_RNA_retina,t_RNA_salivary_gland,t_RNA_seminal_vesicle,t_RNA_skeletal_muscle,t_RNA_skin_1,t_RNA_small_intestine,t_RNA_smooth_muscle,t_RNA_spinal_cord,t_RNA_spleen,t_RNA_stomach_1,t_RNA_testis,t_RNA_thalamus,t_RNA_thymus,t_RNA_thyroid_gland,t_RNA_tongue,t_RNA_tonsil,t_RNA_urinary_bladder,t_RNA_vagina,t_RNA_B-cells,t_RNA_dendritic_cells,t_RNA_granulocytes,t_RNA_monocytes,t_RNA_NK-cells,t_RNA_T-cells,t_RNA_total_PBMC,cell_RNA_A-431,cell_RNA_A549,cell_RNA_AF22,cell_RNA_AN3-CA,cell_RNA_ASC_diff,cell_RNA_ASC_TERT1,cell_RNA_BEWO,cell_RNA_BJ,cell_RNA_BJ_hTERT+,cell_RNA_BJ_hTERT+_SV40_Large_T+,cell_RNA_BJ_hTERT+_SV40_Large_T+_RasG12V,cell_RNA_CACO-2,cell_RNA_CAPAN-2,cell_RNA_Daudi,cell_RNA_EFO-21,cell_RNA_fHDF/TERT166,cell_RNA_HaCaT,cell_RNA_HAP1,cell_RNA_HBEC3-KT,cell_RNA_HBF_TERT88,cell_RNA_HDLM-2,cell_RNA_HEK_293,cell_RNA_HEL,cell_RNA_HeLa,cell_RNA_Hep_G2,cell_RNA_HHSteC,cell_RNA_HL-60,cell_RNA_HMC-1,cell_RNA_HSkMC,cell_RNA_hTCEpi,cell_RNA_hTEC/SVTERT24-B,cell_RNA_hTERT-HME1,cell_RNA_HUVEC_TERT2,cell_RNA_K-562,cell_RNA_Karpas-707,cell_RNA_LHCN-M2,cell_RNA_MCF7,cell_RNA_MOLT-4,cell_RNA_NB-4,cell_RNA_NTERA-2,cell_RNA_PC-3,cell_RNA_REH,cell_RNA_RH-30,cell_RNA_RPMI-8226,cell_RNA_RPTEC_TERT1,cell_RNA_RT4,cell_RNA_SCLC-21H,cell_RNA_SH-SY5Y,cell_RNA_SiHa,cell_RNA_SK-BR-3,cell_RNA_SK-MEL-30,cell_RNA_T-47d,cell_RNA_THP-1,cell_RNA_TIME,cell_RNA_U-138_MG,cell_RNA_U-2_OS,cell_RNA_U-2197,cell_RNA_U-251_MG,cell_RNA_U-266/70,cell_RNA_U-266/84,cell_RNA_U-698,cell_RNA_U-87_MG,cell_RNA_U-937,cell_RNA_WM-115,blood_RNA_basophil,blood_RNA_classical_monocyte,blood_RNA_eosinophil,blood_RNA_gdT-cell,blood_RNA_intermediate_monocyte,blood_RNA_MAIT_T-cell,blood_RNA_memory_B-cell,blood_RNA_memory_CD4_T-cell,blood_RNA_memory_CD8_T-cell,blood_RNA_myeloid_DC,blood_RNA_naive_B-cell,blood_RNA_naive_CD4_T-cell,blood_RNA_naive_CD8_T-cell,blood_RNA_neutrophil,blood_RNA_NK-cell,blood_RNA_non-classical_monocyte,blood_RNA_plasmacytoid_DC,blood_RNA_T-reg,blood_RNA_total_PBMC,brain_RNA_amygdala,brain_RNA_basal_ganglia,brain_RNA_cerebellum,brain_RNA_cerebral_cortex,brain_RNA_hippocampal_formation,brain_RNA_hypothalamus,brain_RNA_midbrain,brain_RNA_olfactory_region,brain_RNA_pons_and_medulla,brain_RNA_thalamus&format=tsv)) via [API](https://www.proteinatlas.org/about/help/dataaccess) and median gene-level TPM by tissue for all genes that are not protein-coding ([`GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct`](https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz)) in order to create mappings between cell and tissue type strings to the Uber-Anatomy, Cell Ontology, and Cell Line Ontology concepts (see [human-protein-atlas](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#human-protein-atlas) for details on the mapping process). The mappings are then used to create the following edge types:  
- rna-cell line  
- rna-tissue type   
- protein-cell line  
- protein-tissue type  


**Output:**  
- All HPA tissue and cell type strings ➞ `HPA_tissues.txt`  
- Mapping HPA strings to ontology concepts (documentation) ➞ `zooma_tissue_cell_mapping_04JAN2020.xlsx` 
- Final HPA-ontology mappings ➞ `HPA_GTEx_TISSUE_CELL_MAP.txt`
- HPA Edges ➞ `HPA_GTEX_RNA_GENE_PROTEIN_EDGES.txt`

***
**Human Protein Atlas**  
To expedite the mapping process, all HPA tissues, cells, cell lines, and fluid types are extracted from the HPA data columns.

In [ ]:
# download data
url = 'https://www.proteinatlas.org/api/search_download.php?search=&columns=g,eg,up,pe,rnatsm,rnaclsm,rnacasm,rnabrsm,rnabcsm,rnablsm,scl,t_RNA_adipose_tissue,t_RNA_adrenal_gland,t_RNA_amygdala,t_RNA_appendix,t_RNA_basal_ganglia,t_RNA_bone_marrow,t_RNA_breast,t_RNA_cerebellum,t_RNA_cerebral_cortex,t_RNA_cervix,_uterine,t_RNA_colon,t_RNA_corpus_callosum,t_RNA_ductus_deferens,t_RNA_duodenum,t_RNA_endometrium_1,t_RNA_epididymis,t_RNA_esophagus,t_RNA_fallopian_tube,t_RNA_gallbladder,t_RNA_heart_muscle,t_RNA_hippocampal_formation,t_RNA_hypothalamus,t_RNA_kidney,t_RNA_liver,t_RNA_lung,t_RNA_lymph_node,t_RNA_midbrain,t_RNA_olfactory_region,t_RNA_ovary,t_RNA_pancreas,t_RNA_parathyroid_gland,t_RNA_pituitary_gland,t_RNA_placenta,t_RNA_pons_and_medulla,t_RNA_prostate,t_RNA_rectum,t_RNA_retina,t_RNA_salivary_gland,t_RNA_seminal_vesicle,t_RNA_skeletal_muscle,t_RNA_skin_1,t_RNA_small_intestine,t_RNA_smooth_muscle,t_RNA_spinal_cord,t_RNA_spleen,t_RNA_stomach_1,t_RNA_testis,t_RNA_thalamus,t_RNA_thymus,t_RNA_thyroid_gland,t_RNA_tongue,t_RNA_tonsil,t_RNA_urinary_bladder,t_RNA_vagina,t_RNA_B-cells,t_RNA_dendritic_cells,t_RNA_granulocytes,t_RNA_monocytes,t_RNA_NK-cells,t_RNA_T-cells,t_RNA_total_PBMC,cell_RNA_A-431,cell_RNA_A549,cell_RNA_AF22,cell_RNA_AN3-CA,cell_RNA_ASC_diff,cell_RNA_ASC_TERT1,cell_RNA_BEWO,cell_RNA_BJ,cell_RNA_BJ_hTERT+,cell_RNA_BJ_hTERT+_SV40_Large_T+,cell_RNA_BJ_hTERT+_SV40_Large_T+_RasG12V,cell_RNA_CACO-2,cell_RNA_CAPAN-2,cell_RNA_Daudi,cell_RNA_EFO-21,cell_RNA_fHDF/TERT166,cell_RNA_HaCaT,cell_RNA_HAP1,cell_RNA_HBEC3-KT,cell_RNA_HBF_TERT88,cell_RNA_HDLM-2,cell_RNA_HEK_293,cell_RNA_HEL,cell_RNA_HeLa,cell_RNA_Hep_G2,cell_RNA_HHSteC,cell_RNA_HL-60,cell_RNA_HMC-1,cell_RNA_HSkMC,cell_RNA_hTCEpi,cell_RNA_hTEC/SVTERT24-B,cell_RNA_hTERT-HME1,cell_RNA_HUVEC_TERT2,cell_RNA_K-562,cell_RNA_Karpas-707,cell_RNA_LHCN-M2,cell_RNA_MCF7,cell_RNA_MOLT-4,cell_RNA_NB-4,cell_RNA_NTERA-2,cell_RNA_PC-3,cell_RNA_REH,cell_RNA_RH-30,cell_RNA_RPMI-8226,cell_RNA_RPTEC_TERT1,cell_RNA_RT4,cell_RNA_SCLC-21H,cell_RNA_SH-SY5Y,cell_RNA_SiHa,cell_RNA_SK-BR-3,cell_RNA_SK-MEL-30,cell_RNA_T-47d,cell_RNA_THP-1,cell_RNA_TIME,cell_RNA_U-138_MG,cell_RNA_U-2_OS,cell_RNA_U-2197,cell_RNA_U-251_MG,cell_RNA_U-266/70,cell_RNA_U-266/84,cell_RNA_U-698,cell_RNA_U-87_MG,cell_RNA_U-937,cell_RNA_WM-115,blood_RNA_basophil,blood_RNA_classical_monocyte,blood_RNA_eosinophil,blood_RNA_gdT-cell,blood_RNA_intermediate_monocyte,blood_RNA_MAIT_T-cell,blood_RNA_memory_B-cell,blood_RNA_memory_CD4_T-cell,blood_RNA_memory_CD8_T-cell,blood_RNA_myeloid_DC,blood_RNA_naive_B-cell,blood_RNA_naive_CD4_T-cell,blood_RNA_naive_CD8_T-cell,blood_RNA_neutrophil,blood_RNA_NK-cell,blood_RNA_non-classical_monocyte,blood_RNA_plasmacytoid_DC,blood_RNA_T-reg,blood_RNA_total_PBMC,brain_RNA_amygdala,brain_RNA_basal_ganglia,brain_RNA_cerebellum,brain_RNA_cerebral_cortex,brain_RNA_hippocampal_formation,brain_RNA_hypothalamus,brain_RNA_midbrain,brain_RNA_olfactory_region,brain_RNA_pons_and_medulla,brain_RNA_thalamus&format=tsv'
if not os.path.exists(unprocessed_data_location + 'proteinatlas_search.tsv'):
    data_downloader(url, unprocessed_data_location, 'proteinatlas_search.tsv.gz')

# load data
hpa = pandas.read_csv(unprocessed_data_location + 'proteinatlas_search.tsv', header=0, delimiter='\t')
hpa.fillna('None', inplace=True)

In [ ]:
# retrieve terms to map and write results
with open(unprocessed_data_location + 'HPA_tissues.txt', 'w') as outfile:
    for x in tqdm(list(hpa.columns)):
        if x.endswith('[NX]'):
            outfile.write(x.split('RNA - ')[-1].split(' [NX]')[:-1][0] + '\n')

***
**Genotype-Tissue Expression Project**  
Import the tissues, cells, cell lines, and fluids that we externally mapped from HPA and GTEx data to [UBERON](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#uber-anatomy-ontology), the [Cell Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#cell-ontology), and the [Cell Line Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#cell-line-ontology).

In [ ]:
# load data
url='https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz'
if not os.path.exists(unprocessed_data_location + 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct'):
    data_downloader(url, unprocessed_data_location)

# load data
gtex = pandas.read_csv(unprocessed_data_location + 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct', header=0, skiprows=2, delimiter='\t')
gtex.fillna('None', inplace=True)  # replace NaN with 'None'
gtex['Name'].replace('(\..*)','', inplace=True, regex=True)  # remove identifier type, which appears after '.'


In [ ]:
# download data
url='https://zenodo.org/records/10056198/files/zooma_tissue_cell_mapping_04JAN2020.xlsx.zip?download=1'
if not os.path.exists(unprocessed_data_location + 'zooma_tissue_cell_mapping_04JAN2020.xlsx'):
    data_downloader(url, unprocessed_data_location)
    
# load ontology mapping data
mapping_data = pandas.read_excel(open(unprocessed_data_location + 'zooma_tissue_cell_mapping_04JAN2020.xlsx', 'rb'),
                                 sheet_name='Concept_Mapping - 04JAN2020', header=0, engine='openpyxl')
mapping_data.fillna('None', inplace=True)  # convert NaN to None

# preview data
mapping_data.head(n=3)

_Write HPA and GTEx Mapping Data_  
The HPA and GTEx mapping data is written locally so that it can be used by the `PheKnowLator` algorithm when creating the knowledge graph edge lists. 

In [ ]:
with open(processed_data_location + 'HPA_GTEx_TISSUE_CELL_MAP.txt', 'w') as out:
    for idx, row in tqdm(mapping_data.iterrows(), total=mapping_data.shape[0]):
        if row['UBERON'] != 'None': out.write(str(row['TERM']).strip() + '\t' + str(row['UBERON']).strip() + '\n')
        if row['CL'] != 'None': out.write(str(row['TERM']).strip() + '\t' + str(row['CL']).strip() + '\n')
        if row['CLO'] != 'None': out.write(str(row['TERM']).strip() + '\t' + str(row['CLO']).strip() + '\n')

In [ ]:
# load mapping data
mapping_data = pandas.read_csv(processed_data_location + 'HPA_GTEx_TISSUE_CELL_MAP.txt', header=None, names=['TISSUE_CELL_TERM', 'ONTOLOGY_IDs'], delimiter='\t')

# preview data
mapping_data.head(n=3)

***

**Create Edge Data Set**

_Human Protein Atlas_  
The `HPA` data is looped over and reformatted such that all tissue, cell, cell lines, and fluid types are stored as a nested list. The anatomy type is specified as an item in the list according to its type in order to make mapping more efficient while building the knowledge graph edge list.

In [ ]:
hpa_results = []
for idx, row in tqdm(hpa.iterrows(), total=hpa.shape[0]):
    ens, gene, uniprot, evid = str(row['Ensembl']), str(row['Gene']), str(row['Uniprot']), str(row['Evidence'])
    if row['RNA tissue specific NX'] != 'None':
        for x in row['RNA tissue specific NX'].split(';'):
            hpa_results += [[ens, gene, uniprot, evid, 'anatomy', str(x.split(':')[0])]]
    if row['RNA cell line specific NX'] != 'None':
        for x in row['RNA cell line specific NX'].split(';'):
            hpa_results += [[ens, gene, uniprot, evid, 'cell line', str(x.split(':')[0])]]
    if row['RNA brain regional specific NX'] != 'None':
        for x in row['RNA brain regional specific NX'].split(';'):
            hpa_results += [[ens, gene, uniprot, evid, 'anatomy', str(x.split(':')[0])]]
    if row['RNA blood cell specific NX'] != 'None':
        for x in row['RNA blood cell specific NX'].split(';'):
            hpa_results += [[ens, gene, uniprot, evid, 'anatomy', str(x.split(':')[0])]]
    if row['RNA blood lineage specific NX'] != 'None':
        for x in row['RNA blood lineage specific NX'].split(';'):
            hpa_results += [[ens, gene, uniprot, evid, 'anatomy', str(x.split(':')[0])]]

_Genotype-Tissue Expression Project_  
The `GTEx` edge data is created by first filtering out all _protein-coding_ genes that appear in the `HPA` cell transcriptome data set. Once filter so that we are only left noncoding genes, we perform an additional filtering step to only add genes and their corresponding tissue, cell, or fluid, if the median expression is `>= 1.0`. The `GTEx` is formatted such that all anatomical entities occur as their own column and all unique genes occur as a row, thus the expression filtering step is performed while also reformatting the file. The genes and tissues/cells/fluids that meet criteria are stored as a nested list.

In [ ]:
# remove rows that contain protein coding genes already in the hpa data
hpa_genes = list(hpa['Ensembl'].drop_duplicates(keep='first', inplace=False))
gtex = gtex.loc[gtex['Name'].apply(lambda x: x not in hpa_genes)]

# loop over data and re-organize - only keep results with tpm >= 1 and if gene symbol is not a protein-coding gene
gtex_results = []
for idx, row in tqdm(gtex.iterrows(), total=gtex.shape[0]):
    for col in list(gtex.columns)[2:]:
        typ = 'cell line' if 'Cells' in col else 'anatomy'
        if row[col] >= 1.0:
            evidence = 'Evidence at transcript level'
            gtex_results += [[str(row['Name']), str(row['Description']), 'None', evidence, typ, str(col)]]

*Writes Edge Data*  

In [ ]:
with open(processed_data_location + 'HPA_GTEX_RNA_GENE_PROTEIN_EDGES.txt', 'w') as out:
    for x in tqdm(hpa_results + gtex_results):
        out.write(x[0] + '\t' + x[1] + '\t' + x[2] + '\t' + x[3] + '\t' + x[4] + '\t' + x[5] + '\n')

In [ ]:
# load data, return edge count, and preview it
hpa_edges = pandas.read_csv(processed_data_location + 'HPA_GTEX_RNA_GENE_PROTEIN_EDGES.txt',
                           header=None, low_memory=False, sep='\t',
                           names=['Ensembl_IDs', 'Gene_Symbols', 'Uniprot_IDs', 'Evidence', 'Anatomy_Type', 'Anatomy'])

print('There are {edge_count} edges'.format(edge_count=len(hpa_edges)))
hpa_edges.head(n=5)

<br>

***

### Mapping Reactome Pathways to the Pathway Ontology <a class="anchor" id="reactome-pw"></a>

**Data Source Wiki Page:** [Pathway Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#pathway-ontology)  

**Purpose:** This script downloads the [canonical pathways](http://compath.scai.fraunhofer.de/export_mappings) and [kegg-reactome pathway mappings](https://github.com/ComPath/resources/blob/master/mappings/kegg_reactome.csv) files from the [ComPath Ecosystem](https://github.com/ComPath) in order to create the following identifier mappings:  
- `Reactome Pathway Identifiers`  ➞ `KEGG Pathway Identifiers` ➞ `Pathway Ontology Identifiers` 

**Output:**  
- `REACTOME_PW_GO_MAPPINGS.txt`


***

**Pathway Ontology**   
Use [OWL Tools](https://github.com/owlcollab/owltools/wiki) to download the [Pathway Ontology](http://www.obofoundry.org/ontology/pw.html). Once downloaded, we read the ontology in as a `RDFLib` graph object so that we can query it to obtain all `DbXRefs`.

In [ ]:
# download ontology
if not os.path.exists(unprocessed_data_location + 'pw_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/pw.owl',
                             unprocessed_data_location + 'pw_with_imports.owl'))

# load the knowledge graph
pw_graph = Graph().parse(unprocessed_data_location + 'pw_with_imports.owl')
print('There are {} axioms in the ontology (date: {})'.format(len(pw_graph), datetime.datetime.now().strftime('%m/%d/%Y')))

_Reformat Mapping Results_  
Create a dictionary of mapping results where pathway ontology identifiers are values and the keys are `DbXRef` identifiers.


In [ ]:
# get dbxref results
dbxref_res = gets_ontology_class_dbxrefs(pw_graph)[0]
dbxref_dict = {str(k).lower().split('/')[-1]: {str(i).split('/')[-1].replace('_', ':') for i in v} for k, v in dbxref_res.items() if 'PW_' in str(v)}

# get synonym results
syn_res = gets_ontology_class_synonyms(pw_graph)[0]
synonym_dict = {str(k).lower().split('/')[-1]: {str(i).split('/')[-1].replace('_', ':') for i in v} for k, v in syn_res.items() if 'PW_' in str(v)}

# combine results into single dictionary
id_mappings = {**dbxref_dict, **synonym_dict}

print('There are {} results (date: {})'.format(len(id_mappings), datetime.datetime.now().strftime('%m/%d/%Y')))

***

**Reactome Pathways**  
Download a file of all [Reactome Pathways](https://reactome.org/download/current/ReactomePathways.txt), [Reactome's GO Annotations]('https://reactome.org/download/current/gene_association.reactome.gz'), and [Reactome's mappings to CHEBI](https://reactome.org/download/current/ChEBI2Reactome_All_Levels.txt). This file will be filtered to only include human pathways.

_Reactome Pathway Stable Identifiers_

In [ ]:
# download data
url = 'https://reactome.org/download/current/ReactomePathways.txt'
if not os.path.exists(unprocessed_data_location + 'ReactomePathways.txt'):
    data_downloader(url, unprocessed_data_location)

# load data
reactome_pathways = pandas.read_csv(unprocessed_data_location + 'ReactomePathways.txt', header=None, delimiter='\t', low_memory=False)

In [ ]:
# remove all non-human pathways and save as list
reactome_pathways = reactome_pathways.loc[reactome_pathways[2].apply(lambda x: x == 'Homo sapiens')] 
reactome_map = {x:set(['PW_0000001']) for x in set(list(reactome_pathways[0]))}     

_Reactome's Mappings to GO Annotations_

In [ ]:
# download data
url = 'https://reactome.org/download/current/gene_association.reactome.gz'
if not os.path.exists(unprocessed_data_location + 'gene_association.reactome'):
    data_downloader(url, unprocessed_data_location)

# load data
reactome_pathways2 = pandas.read_csv(unprocessed_data_location + 'gene_association.reactome', header=None, delimiter='\t', skiprows=3, low_memory=False)

In [ ]:
# remove all non-human pathways and save as list
reactome_pathways2 = reactome_pathways2.loc[reactome_pathways2[12].apply(lambda x: x == 'taxon:9606')] 
reactome_map.update({x.split(':')[-1]:set(['PW_0000001']) for x in set(list(reactome_pathways2[5]))})     

_Reactome's Mappings to ChEBI_

In [ ]:
# download data
url = 'https://reactome.org/download/current/ChEBI2Reactome_All_Levels.txt'
if not os.path.exists(unprocessed_data_location + 'ChEBI2Reactome_All_Levels.txt'):
    data_downloader(url, unprocessed_data_location)

# load data
reactome_pathways3 = pandas.read_csv(unprocessed_data_location + 'ChEBI2Reactome_All_Levels.txt', header=None, delimiter='\t', low_memory=False)

In [ ]:
# remove all non-human pathways and save as list
reactome_pathways3 = reactome_pathways3.loc[reactome_pathways3[5].apply(lambda x: x == 'Homo sapiens')] 
reactome_map.update({x:set(['PW_0000001']) for x in set(list(reactome_pathways3[1]))})     

***

**ComPath Reactome Pathway Mappings**  
Use [ComPath Mappings](https://github.com/ComPath/resources/tree/master/mappings) to obtain the following mappings:  `Reactome Pathways`  ➞ `KEGG Pathways` ➞ `Pathway Ontology` 

_Canonical Pathways_

In [ ]:
# download data
url1 = 'http://compath.scai.fraunhofer.de/export_mappings'
if not os.path.exists(unprocessed_data_location + 'compath_canonical_pathway_mappings.txt'):
    data_downloader(url1, unprocessed_data_location, 'compath_canonical_pathway_mappings.txt')

# load data
compath_cannonical = pandas.read_csv(unprocessed_data_location + 'compath_canonical_pathway_mappings.txt', header=None, delimiter='\t', low_memory=False)
compath_cannonical.fillna('None', inplace=True)

In [ ]:
for idx, row in tqdm(compath_cannonical.iterrows(), total=compath_cannonical.shape[0]):
    if row[6] == 'kegg' and 'kegg:' + row[5].strip('path:hsa') in id_mappings.keys() and row[2] == 'reactome':
        for x in id_mappings['kegg:' + row[5].strip('path:hsa')]:
            if row[1] in reactome_map.keys(): reactome_map[row[1]] |= set([x.split('/')[-1]])
            else: reactome_map[row[1]] = set([x.split('/')[-1]])
    if (row[2] == 'kegg' and 'kegg:' + row[1].strip('path:hsa') in id_mappings.keys()) and row[6] == 'reactome':
        for x in id_mappings['kegg:' + row[1].strip('path:hsa')]:
            if row[5] in reactome_map.keys(): reactome_map[row[5]] |= set([x.split('/')[-1]])
            else: reactome_map[row[5]] = set([x.split('/')[-1]])         

_KEGG - Reactome Mappings_

In [ ]:
# download data
url2 = 'https://raw.githubusercontent.com/ComPath/resources/master/mappings/kegg_reactome.csv'
if not os.path.exists(unprocessed_data_location + 'kegg_reactome.csv'):
    data_downloader(url2, unprocessed_data_location, 'kegg_reactome.csv')

# load data
kegg_reactome_map = pandas.read_csv(unprocessed_data_location + 'kegg_reactome.csv', header=0, delimiter=',', low_memory=False)

In [ ]:
for idx, row in tqdm(kegg_reactome_map.iterrows(), total=kegg_reactome_map.shape[0]):
    if row['Source Resource'] == 'reactome' and 'kegg:' + row['Target ID'].strip('path:hsa') in id_mappings.keys():
        for x in id_mappings['kegg:' + row['Target ID'].strip('path:hsa')]:
            if row['Source ID'] in reactome_map.keys(): reactome_map[row['Source ID']] |= set([x.split('/')[-1]])
            else: reactome_map[row['Source ID']] = set([x.split('/')[-1]])
    if row['Target Resource'] == 'reactome' and 'kegg:' + row['Source Resource'].strip('path:hsa') in id_mappings.keys():
        for x in id_mappings['kegg:' + row['Source ID'].strip('path:hsa')]:
            if row['Target ID'] in reactome_map.keys(): reactome_map[row['Target ID']] |= set([x.split('/')[-1]])
            else: reactome_map[row['Target ID']] = set([x.split('/')[-1]])

***

**Reactome Pathway GO Annotation Mappings**  
Use Reactome's [API](https://reactome.org/dev/content-service) to obtain the following mappings: `Reactome Pathway Identifiers`  ➞ `Gene Ontology Identifiers`.

In [ ]:
for request_ids in tqdm(list(chunks(list(reactome_map.keys()), 20))):
    result, key = content.query_ids(ids=','.join(request_ids)), 'goBiologicalProcess'
    if result is not None and (isinstance(result, List) or result['code'] != 404):
        for res in result:
            if key in res.keys():
                if res['stId'] in reactome_map.keys(): reactome_map[res['stId']] |= {'GO_' + res[key]['accession']}
                else: reactome_map[res['stId']] = {'GO_' + res[key]['accession']}

*Write Data*

In [ ]:
# reformat identifiers -- replacing ontology concepts with ':' to '_'
temp_dict = dict()
for key, value in tqdm(reactome_map.items()):
    temp_dict[key] = set(x.replace(':', '_') for x in value)

# overwrite original reactome dict with cleaned mappings
reactome_map = temp_dict

# output data
with open(processed_data_location + 'REACTOME_PW_GO_MAPPINGS.txt', 'w') as out:
    for key in tqdm(reactome_map.keys()):
        for x in reactome_map[key]:
            if x.startswith('PW') or x.startswith('GO'): out.write(key + '\t' + x + '\n')

In [ ]:
# load data, print row count, and preview it
pw_data = pandas.read_csv(processed_data_location + 'REACTOME_PW_GO_MAPPINGS.txt', header=None, names=['Pathway_IDs', 'Mapping_IDs'], delimiter='\t')

print('There are {edge_count} pathway ontology mappings'.format(edge_count=len(pw_data)))
pw_data.head(n=5)

<br>

***

### Mapping Genomic Identifiers to the Sequence Ontology <a class="anchor" id="genomic-soo"></a>

**Data Source Wiki Page:** [Sequence Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/_edit#sequence-ontology)  

**Purpose:** This script downloads the `genomic_sequence_ontology_mappings.xlsx` file in order to create the following identifier mappings:  
- `Gene BioTypes`  ➞ `Sequence Ontology Identifiers`  
- `RNA BioTypes`  ➞ `Sequence Ontology Identifiers`  
- `variant Types`  ➞ `Sequence Ontology Identifiers`

**Output:**  
- `SO_GENE_TRANSCRIPT_VARIANT_TYPE_MAPPING.txt`


In [ ]:
# download data
url='https://zenodo.org/records/10056198/files/genomic_sequence_ontology_mappings.xlsx.zip?download=1'
if not os.path.exists(unprocessed_data_location + 'genomic_sequence_ontology_mappings.xlsx'):
    data_downloader(url, unprocessed_data_location)

# load data
mapping_data = pandas.read_excel(open(unprocessed_data_location + 'genomic_sequence_ontology_mappings.xlsx', 'rb'),
                                 sheet_name='GenomicType_SO_Map_09Mar2020', header=0, engine='openpyxl')

# convert data to dictionary
genomic_type_so_map = {}
for idx, row in tqdm(mapping_data.iterrows(), total=mapping_data.shape[0]):
    genomic_type_so_map[row['source_*_type'] + '_' + row['Genomic']] = row['SO ID']

***

**Genes**

In [ ]:
# read in genomic mapping data
genomic_mapped_ids = pickle.load(open(processed_data_location + 'Merged_gene_rna_protein_identifiers.pkl', 'rb'))

sequence_map = {}
for identifier in tqdm(genomic_mapped_ids.keys()):    
    if identifier.startswith('entrez_id_') and identifier.replace('entrez_id_', '') != 'None':
        id_clean = identifier.replace('entrez_id_', '')
        
        # get identifier types
        ensembl = [x.replace('ensembl_gene_type_', '') for x in genomic_mapped_ids[identifier] if x.startswith('ensembl_gene_type') and x != 'ensembl_gene_type_unknown']
        hgnc = [x.replace('hgnc_gene_type_', '')  for x in genomic_mapped_ids[identifier] if x.startswith('hgnc_gene_type') and x != 'hgnc_gene_type_unknown']
        entrez = [x.replace('entrez_gene_type_', '')  for x in genomic_mapped_ids[identifier] if x.startswith('entrez_gene_type') and x != 'entrez_gene_type_unknown']
        
        # determine gene type
        if len(ensembl) > 0: gene_type = genomic_type_so_map[ensembl[0].replace('ensembl_gene_type_', '') + '_Gene']
        elif len(hgnc) > 0: gene_type = genomic_type_so_map[hgnc[0].replace('hgnc_gene_type_', '') + '_Gene']
        elif len(entrez) > 0: gene_type = genomic_type_so_map[entrez[0].replace('entrez_gene_type_', '') + '_Gene']
        else: gene_type = 'SO_0000704'  
        
        # update sequence map
        if id_clean in sequence_map.keys(): sequence_map[id_clean] += [gene_type]
        else: sequence_map[id_clean] = [gene_type]

***

**Transcripts**

In [ ]:
# read in processed Ensembl Transcript data 
transcript_data = pandas.read_csv(processed_data_location + 'ensembl_identifier_data_cleaned.txt', header=0, delimiter='\t', low_memory=False)

# convert to dictionary
transcripts = {}
for idx, row in tqdm(transcript_data.iterrows(), total=transcript_data.shape[0]):
    if row['transcript_stable_id'] != 'None':
        if row['transcript_stable_id'].replace('transcript_stable_id_', '') in transcripts.keys():
            transcripts[row['transcript_stable_id'].replace('transcript_stable_id_', '')] += [row['ensembl_transcript_type']]
        else: transcripts[row['transcript_stable_id'].replace('transcript_stable_id_', '')] = [row['ensembl_transcript_type']]
            
# update so map dictionary
for identifier in tqdm(transcripts.keys()):
    if transcripts[identifier][0] == 'protein_coding': trans_type = genomic_type_so_map['protein-coding_Transcript']
    elif transcripts[identifier][0] == 'misc_RNA': trans_type = genomic_type_so_map['miscRNA_Transcript']
    else: trans_type = genomic_type_so_map[list(set(transcripts[identifier]))[0] + '_Transcript']
    sequence_map[identifier] = [trans_type, 'SO_0000673']

***

**Variants**

In [ ]:
# read in variant summary data 
url = 'ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz'
if not os.path.exists(unprocessed_data_location + 'variant_summary.txt'):
    data_downloader(url, unprocessed_data_location)
    
# load data    
variant_data = pandas.read_csv(unprocessed_data_location + 'variant_summary.txt', header=0, delimiter='\t', low_memory=False)

# convert to dictionary
variants = {}
for idx, row in tqdm(variant_data.iterrows(), total=variant_data.shape[0]):
    if row['Assembly'] == 'GRCh38' and row['RS# (dbSNP)'] != -1:
        if 'rs' + str(row['RS# (dbSNP)']) in variants.keys(): variants['rs' + str(row['RS# (dbSNP)'])] |= set([row['Type']])
        else: variants['rs' + str(row['RS# (dbSNP)'])] = set([row['Type']])

# update so map dictionary
for identifier in tqdm(variants.keys()):
    for typ in variants[identifier]:
        var_type = genomic_type_so_map[typ.lower() + '_Variant']
        if identifier in sequence_map.keys(): sequence_map[identifier] += [var_type]
        else: sequence_map[identifier] = [var_type]

*** 
**Write Data**

In [ ]:
# reformat data and write it out
with open(processed_data_location + 'SO_GENE_TRANSCRIPT_VARIANT_TYPE_MAPPING.txt', 'w') as outfile:
    for key in tqdm(sequence_map.keys()):
        for map_type in sequence_map[key]:
            outfile.write(key + '\t' + map_type + '\n')

# load data, print row count, and preview it
so_data = pandas.read_csv(processed_data_location + 'SO_GENE_TRANSCRIPT_VARIANT_TYPE_MAPPING.txt', header=None, delimiter='\t', names=['Identifier', 'Sequence_Ontology_ID'])

print('There are {edge_count} sequence ontology mappings'.format(edge_count=len(so_data)))
so_data.head(n=5)

***

**Combine Pathway and Sequence Ontology Mapping Data in Dictionary**  
Combine the pathway and sequence mapping data into a dictionary and output it.

In [ ]:
# combine genomic and pathway maps
subclass_mapping = {}  
sequence_map.update(reactome_map)

# iterate over pathway lists and combine them
for key in tqdm(sequence_map.keys()):
    subclass_mapping[key] = sequence_map[key]

# save a copy of the dictionary
pickle.dump(subclass_mapping, open(construction_approach_location + 'subclass_construction_map.pkl', 'wb'), protocol=4)

<br>

***
***
### CREATE EDGE DATASETS  <a class="anchor" id="create-edge-datasets"></a>
***
***

### Ontologies  <a class="anchor" id="ontologies"></a>
***
- [Protein Ontology](#protein-ontology)  
- [Relations Ontology](#relations-ontology)  

***
***

***
### Protein Ontology <a class="anchor" id="protein-ontology"></a>

**Data Source Wiki Page:** [protein-ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#human-phenotype-ontology)  

**Purpose:** This script uses [OWLTools](https://github.com/owlcollab/owltools) to download the [pr.owl](http://purl.obolibrary.org/obo/pr.owl) (with imports) file from [ProConsortium.org](https://proconsortium.org/) in order to create a version of the ontology that contains only human proteins. This is achieved by performing forward and reverse breadth first search over all proteins which are `owl:subClassOf` [Homo sapiens protein](https://proconsortium.org/app/entry/PR%3A000029067/).

<br>

**Output:**  
- Human Protein Ontology ➞ `human_pro.owl`
- Classified Human Protein Ontology (Hermit) ➞ `human_pro_closed.owl`


In [ ]:
# download ontology
if not os.path.exists(unprocessed_data_location + 'pr_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/pr.owl',
                             unprocessed_data_location + 'pr_with_imports.owl'))
    
# read in ontology as graph (the ontology is large so this takes ~60 minutes)
print('Loading Protein Ontology')
pr_graph = Graph().parse(unprocessed_data_location + 'pr_with_imports.owl')
print('There are {} axioms in the ontology (date: {})'.format(len(pr_graph), datetime.datetime.now().strftime('%m/%d/%Y')))

_Convert Ontology to Directed MulitGraph_  
In order to create a version of the ontology which includes all relevant human edges, we need to first convert the KG to a [directed multigraph](https://networkx.github.io/documentation/stable/reference/classes/multidigraph.html).

In [ ]:
networkx_mdg: networkx.MultiDiGraph = networkx.MultiDiGraph()
    
for s, p, o in tqdm(pr_graph):
    networkx_mdg.add_edge(s, o, **{'key': p})

_Identify Human Proteins_   
A list of human proteins is obtained by querying the ontology to return all ontology classes `only_in_taxon some Homo sapiens`.

*Approach 1 - Query Loaded Graph to Obtain Human Protein Classes*  
Does not require using external resources or SPARQL Endpoints. This is the preferred approach.

In [ ]:
human_classes_restriction = list(pr_graph.triples((None, OWL.someValuesFrom, obo.NCBITaxon_9606)))
human_classes = [list(pr_graph.subjects(RDFS.subClassOf, x[0])) for x in human_classes_restriction]
human_pro_classes = list(str(i) for j in human_classes for i in j if 'PR_' in str(i))

print('There are {} edges in the ontology (date:{})'.format(len(human_pro_classes), datetime.datetime.now().strftime('%m/%d/%Y')))

*Approach 2 - Query PRO Consortium SPARQL Endpoint to Obtain Human Protein Classes*  
This approach should only be used when the PRO endpoint is not limiting the number of results that are returned. As of `October 2021`, this was happening so please use *Approach 1* which is guaranteed to return the correct results.

In [ ]:
# # download data
# url = 'https://sparql.proconsortium.org/virtuoso/sparql?query=PREFIX+obo%3A+%3Chttp%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F%3E%0D%0A%0D%0ASELECT+%3FPRO_term%0D%0AFROM+%3Chttp%3A%2F%2Fpurl.obolibrary.org%2Fobo%2Fpr%3E%0D%0AWHERE+%7B%0D%0A+++++++%3FPRO_term+rdf%3Atype+owl%3AClass+.%0D%0A+++++++%3FPRO_term+rdfs%3AsubClassOf+%3Frestriction+.%0D%0A+++++++%3Frestriction+owl%3AonProperty+obo%3ARO_0002160+.%0D%0A+++++++%3Frestriction+owl%3AsomeValuesFrom+obo%3ANCBITaxon_9606+.%0D%0A%0D%0A+++++++%23+use+this+to+filter-out+things+like+hgnc+ids%0D%0A+++++++FILTER+%28regex%28%3FPRO_term%2C%22http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F*%22%29%29+.%0D%0A%7D&format=text%2Fhtml&debug='
# if not os.path.exists(unprocessed_data_location + 'human_pro_classes.html'):
#     data_downloader(url, unprocessed_data_location, 'human_pro_classes.html')

# # load data
# df_list = pandas.read_html(unprocessed_data_location + 'human_pro_classes.html')

# # extract data from html table - pro classes only_in_taxon some Homo sapiens
# human_pro_classes = list(df_list[-1]['PRO_term'])
# print('There are {} edges in the ontology (date:{})'.format(len(human_pro_classes), datetime.datetime.now().strftime('%m/%d/%Y')))

_Construct Human PRO_   
Now that we have all of the paths from the original graph that are relevant to humans, we can construct a human-only version of the PRotein Ontology. After building the human subset, we verify the number of connected components and get 1. However, after reformatting the graph using [OWLTools](https://github.com/owlcollab/owltools) you will see that there are 3 connected components: component 1 (n=`1051673`); component 2 (n=`12`); and component 3 (n=`2`). The contents of components 2 and 3 are shown below:

```python
[{'http://purl.obolibrary.org/obo/IAO_0000115',
  'http://www.geneontology.org/formats/oboInOwl#hasAlternativeId',
  'http://www.geneontology.org/formats/oboInOwl#hasBroadSynonym',
  'http://www.geneontology.org/formats/oboInOwl#hasDbXref',
  'http://www.geneontology.org/formats/oboInOwl#hasExactSynonym',
  'http://www.geneontology.org/formats/oboInOwl#hasNarrowSynonym',
  'http://www.geneontology.org/formats/oboInOwl#hasOBONamespace',
  'http://www.geneontology.org/formats/oboInOwl#hasRelatedSynonym',
  'http://www.geneontology.org/formats/oboInOwl#id',
  'http://www.geneontology.org/formats/oboInOwl#is_transitive',
  'http://www.geneontology.org/formats/oboInOwl#shorthand',
  'http://www.w3.org/2002/07/owl#AnnotationProperty'},
 
 {'N41f0be4cf00c48929605b1e69a09f326',
  'http://www.w3.org/2002/07/owl#Ontology'}]
```

In [ ]:
# create a new graph using bfs paths
human_pro_graph = Graph()
human_networkx_mdg = networkx.MultiDiGraph()

for node in tqdm(human_pro_classes):
    forward = list(networkx.edge_bfs(networkx_mdg, URIRef(node), orientation='original'))
    reverse = list(networkx.edge_bfs(networkx_mdg, URIRef(node), orientation='reverse'))
    
    # add edges from forward and reverse bfs paths
    for path in set(forward + reverse):
        human_pro_graph.add((path[0], path[2], path[1]))
        human_networkx_mdg.add_edge(path[0], path[1], **{'key': path[2]})

In [ ]:
# get connected component information
print('Finding Connected Components')
components = list(networkx.connected_components(human_networkx_mdg.to_undirected()))
component_dict = sorted(components, key=len, reverse=True)

# if more than 1 connected component, only keep the biggest
if len(component_dict) > 1:
    print('Cleaning Graph: Removing Small Disconnected Components')
    for node in tqdm([x for y in component_dict[1:] for x in list(y)]):
        human_pro_graph.remove((node, None, None))

# save data
print('Saving Human Subset of the Protein Ontology')
human_pro_graph.serialize(destination=unprocessed_data_location + 'human_pro.owl', format='xml')

_Classify Ontology_  
To ensure that we have correctly built the new ontology, we run the hermit reasoner over it to ensure that there are no incomplete triples or inconsistent classes. In order to do this, we will call the reasoner using [OWLTools](https://github.com/owlcollab/owltools), which this script assumes has already been downloaded to the `./resources/lib` directory. The following arguments are then called to run the reasoner (from the command line):  

___

```bash
../pkt_kg/libs/owltools ./resources/processed_data/unprocessed_data/human_pro.owl --reasoner elk --run-reasoner --assert-implied -o ./resources/processed_data/human_pro_closed.owl
```
___


_**Note.** This step takes around 5 minutes to run. When run from the command line the reasoner determined that the ontology was consistent and 200 new axioms were inferred (12/01/2020)._

In [ ]:
# run reasoner
command = '{} {} --reasoner {} --run-reasoner --assert-implied -o {}'
os.system(command.format(owltools_location, unprocessed_data_location + 'human_pro.owl', 'elk',
                         ontology_data_location + 'pr_with_imports.owl'))

_Examine Cleaned Human PRO_  
Once we have cleaned the ontology we can get counts of components, nodes, and edges.

In [ ]:
gets_ontology_statistics(ontology_data_location + 'pr_with_imports.owl')

<br>

***

### Relations Ontology <a class="anchor" id="relations-ontology"></a>

**Data Source Wiki Page:** [RO](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#relation-ontology)  

**Purpose:** This script downloads the [ro.owl](http://purl.obolibrary.org/obo/ro.owl) file from [obofoundry.org](http://www.obofoundry.org/) in order to obtain all `ObjectProperties` and their inverse relations.  

**Output:** 
- Relations and Inverse Relations ➞ `INVERSE_RELATIONS.txt`
- Relations and Labels ➞ `RELATIONS_LABELS.txt`

In [ ]:
# download ontology
if not os.path.exists(unprocessed_data_location + 'ro_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/ro.owl',
                             unprocessed_data_location + 'ro_with_imports.owl'))
# load graph
ro_graph = Graph().parse(unprocessed_data_location + 'ro_with_imports.owl')
print('There are {} edges in the ontology (date:{})'.format(len(ro_graph), datetime.datetime.now().strftime('%m/%d/%Y')))

***

**Identify Relations and Inverse Relations**  
Identify all relations and their inverse relations using the `owl:inverseOf` property. To make it easier to look up the inverse relations, each pair is listed twice, for example:  
- [location of](http://www.ontobee.org/ontology/RO?iri=http://purl.obolibrary.org/obo/RO_0001015) `owl:inverseOf` [located in](http://www.ontobee.org/ontology/RO?iri=http://purl.obolibrary.org/obo/RO_0001025)  
- [located in](http://www.ontobee.org/ontology/RO?iri=http://purl.obolibrary.org/obo/RO_0001025) `owl:inverseOf` [location of](http://www.ontobee.org/ontology/RO?iri=http://purl.obolibrary.org/obo/RO_0001015)

In [ ]:
with open(relations_data_location + 'INVERSE_RELATIONS.txt', 'w') as outfile:
    outfile.write('Relation' + '\t' + 'Inverse_Relation' + '\n')
    for s, p, o in tqdm(ro_graph):
        if 'owl#inverseOf' in str(p):
            if 'RO' in str(s) and 'RO' in str(o):
                outfile.write(str(s.split('/')[-1]) + '\t' + str(o.split('/')[-1]) + '\n')
                outfile.write(str(o.split('/')[-1]) + '\t' + str(s.split('/')[-1]) + '\n')

_Preview Processed Data_

In [ ]:
# load data, print row count, and preview it
ro_data = pandas.read_csv(relations_data_location + 'INVERSE_RELATIONS.txt', header=0, delimiter='\t')

print('There are {edge_count} RO Relations and Inverse Relations'.format(edge_count=len(ro_data)))
ro_data.head(n=5)

***

**Get Relations Labels**  
Identify all relations and their labels for use when building the knowledge graph.

In [ ]:
results = {str(x[2]).lower(): str(x[0]) for x in ro_graph if '/RO_' in str(x[0]) and 'label' in str(x[1]).lower()}

# write data to file
with open(relations_data_location + 'RELATIONS_LABELS.txt', 'w') as outfile:
    outfile.write('Label' + '\t' + 'Relation' + '\n')
    for k, v in results.items():
        outfile.write(str(v).split('/')[-1] + '\t' + str(k) + '\n')

_Preview Processed Data_

In [ ]:
# load data, print row count, and preview it
ro_data_label = pandas.read_csv(relations_data_location + 'RELATIONS_LABELS.txt', header=0, delimiter='\t')

print('There are {edge_count} RO Relations and Labels'.format(edge_count=len(ro_data_label)))
ro_data_label.head(n=5)

<br>

***
***
### Linked Data <a class="anchor" id="linked-data"></a>
***
* [Clinvar Variant-Diseases and Phenotypes](#clinvar-variant) 
* [Uniprot Protein-Cofactor and Protein-Catalyst](#uniprot-protein-cofactorcatalyst)  

***

***
***
### Clinvar Variant-Diseases and Phenotypes <a class="anchor" id="clinvar-variant"></a>

**Data Source Wiki Page:** [Clinvar](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#clinvar)  

**Purpose:** This script downloads the [variant_summary.txt](ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz) file from [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/) in order to create the following edges:  
- gene-variant  
- variant-disease  
- variant-phenotype  

**Output:** `CLINVAR_VARIANT_GENE_DISEASE_PHENOTYPE_EDGES.txt`


In [ ]:
# download data
url = 'ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz'
if not os.path.exists(unprocessed_data_location + 'variant_summary.txt'):
    data_downloader(url, unprocessed_data_location)

# load data
clinvar_data = pandas.read_csv(unprocessed_data_location + 'variant_summary.txt', header=0, delimiter='\t', low_memory=False)

_Preprocess Data_

In [ ]:
# replace NaN with 'None'
clinvar_data.fillna('None', inplace=True)

# explode nested data
explode_df_clinvar = explodes_data(clinvar_data.copy(), ['PhenotypeIDS'], ';')
explode_df_clinvar = explodes_data(explode_df_clinvar.copy(), ['PhenotypeIDS'], ',')

# edit column formatting
explode_df_clinvar['PhenotypeIDS'].replace('Orphanet:ORPHA','ORPHA:', inplace=True, regex=True)
explode_df_clinvar['PhenotypeIDS'].replace('Human Phenotype Ontology:HP:','HP_', inplace=True, regex=True)

# write data
explode_df_clinvar.to_csv(processed_data_location + 'CLINVAR_VARIANT_GENE_DISEASE_PHENOTYPE_EDGES.txt', header=True, sep='\t', encoding='utf-8', index=False)

# print row count and preview data
print('There are {edge_count} variant edges'.format(edge_count=len(explode_df_clinvar)))
explode_df_clinvar.head(n=5)


<br>

***

### Uniprot  Protein-Cofactor and Protein-Catalyst <a class="anchor" id="uniprot-protein-cofactorcatalyst"></a>

**Data Source Wiki Page:** [Uniprot](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#uniprot-knowledgebase)  

**Purpose:** This script downloads the [uniprot-cofactor-catalyst.tab](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources/#uniprot-knowledgebase) file from the [Uniprot Knowledge Base](https://www.uniprot.org) in order to create the following edges:  
- protein-cofactor  
- protein-catalyst  

**Data:** This data was obtained by querying the [UniProt Knowledgebase](https://www.uniprot.org/uniprot/) using the *reviewed:yes AND organism:"Homo sapiens (Human) [9606]""* keyword and including the following columns:
- Entry (Standard) 
- Status (Standard) 
- PRO (*Miscellaneous*)  
- ChEBI (Cofactor) (*Chemical entities*)   
- ChEBI (Catalytic activity) (*Chemical entities*)  

The URL to access the results of this query is obtained by clicking on the share symbol and copying the free-text from the box. To obtain the data in a tab-delimited format the following string is appended to the end of the URL: "&format=tab".

**NOTE.** Be sure to obtain a new URL from the [UniProt Knowledgebase](https://www.uniprot.org/uniprot/) when rebuilding to ensure you are getting the most up-to-date data. This query was last generated on `12/02/2020`.

<br>

**Output:**  
- protein-cofactor ➞ `UNIPROT_PROTEIN_COFACTOR.txt`
- protein-catalyst ➞ `UNIPROT_PROTEIN_CATALYST.txt`


In [ ]:
# download data
url = 'https://www.uniprot.org/uniprot/?query=&fil=organism%3A%22Homo%20sapiens%20(Human)%20%5B9606%5D%22&columns=id%2Creviewed%2Centry%20name%2Cdatabase(PRO)%2Cchebi(Cofactor)%2Cchebi(Catalytic%20activity)&format=tab'
if not os.path.exists(unprocessed_data_location + 'uniprot-cofactor-catalyst.tab'):
    data_downloader(url, unprocessed_data_location, 'uniprot-cofactor-catalyst.tab')

# upload datta
data = open(unprocessed_data_location + 'uniprot-cofactor-catalyst.tab').readlines()

# reformat data and write it out
with open(processed_data_location + 'UNIPROT_PROTEIN_COFACTOR.txt', 'w') as outfile1, open(processed_data_location + 'UNIPROT_PROTEIN_CATALYST.txt', 'w') as outfile2:
    for line in tqdm(data):
        # get cofactors
        if 'CHEBI' in line.split('\t')[4]: 
            for i in line.split('\t')[4].split(';'):
                chebi = i.split('[')[-1].replace(']', '').replace(':', '_')
                outfile1.write('PR_' + line.split('\t')[3].strip(';') + '\t' + chebi + '\n')
        # get catalysts
        if 'CHEBI' in line.split('\t')[5]:       
            for i in line.split('\t')[5].split(';'):
                chebi = i.split('[')[-1].replace(']', '').replace(':', '_')
                outfile2.write('PR_' + line.split('\t')[3].strip(';') + '\t' + chebi + '\n')

***

**Cofactor Data**  

In [ ]:
# load data, print row count, and preview it
pcp1_data = pandas.read_csv(processed_data_location + 'UNIPROT_PROTEIN_COFACTOR.txt', header=None, names=['Protein_Ontology_IDs', 'CHEBI_IDs'], delimiter='\t')

print('There are {edge_count} protein-cofactor edges'.format(edge_count=len(pcp1_data)))
pcp1_data.head(n=5)

***


**Catalyst Data**  

In [ ]:
# load data, print row count, and preview it
pcp2_data = pandas.read_csv(processed_data_location + 'UNIPROT_PROTEIN_CATALYST.txt', header=None, names=['Protein_Ontology_IDs', 'CHEBI_IDs'], delimiter='\t')

print('There are {edge_count} protein-catalyst edges'.format(edge_count=len(pcp2_data)))
pcp2_data.head(n=5)

<br>

***
***
### INSTANCE AND/OR SUBCLASS (NON-ONTOLOGY CLASS) METADATA <a class="anchor" id="create-instance-metadata"></a>
***

**Data Source Wiki Page:** [Dependencies](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies/#node-metadata) 

**Purpose:** The goal of this section is to obtain metadata for each non-ontology instance and/or subclass data source and all relations used in the knowledge graph. For **[`Release V2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**, the following are non-ontology instance and/or subclass data and require the compiling of metadata:
- [Genes](#gene-metadata)
- [RNA](#rna-metadata)
- [Variants](#variant-metadata)  
- [Pathways](#pathway-metadata)
- [Relations](#relations-metadata)

<br>

**Metadata:** The <u>metadata</u> we will gather includes:  

| **Metadata Type** | **Definition** | **Example Node**  | **Example Node Metadata** | 
| :---: | :---: | :---: | :---: | 
| Label | The primary label or name for the node | `R-HSA-1006173` | "CFH:Host cell surface" |       
| Description | A definition or other useful details about the node | `rs794727058` | This `germline` `single nucleotide variant` located on chromosome `5 (GRCh38: NC_000005.10, start/stop positions (126555930/126555930))` with `pathogenic` clinical significance and a last review date of `2/23/2015` (review status: `criteria provided, single submitter`). |        
| Synonym | Alternative terms used for a node | `81399` | "OR1-1, OR7-21" |           

The metadata information will be used to create the following edges in the knowledge graph:  
- **Label** ➞ node `rdfs:label`  
- **Description** ➞ node `obo:IAO_0000115` description 
- **Synonyms** ➞ node `oboInOwl:hasExactSynonym` synonym 

<br>

*<b>NOTE.</b> All node metadata are written to the `node_data` directory as a `pickled` dictionary called `node_metadata_dict.pkl`. The algorithm will look for this dictionary in the `node_data` directory and if it is not there, then no node metadata will be created.*

<br>

### Prepare Metadata Dictionaries
***

**Purpose:** To create the resources needed in order to create metadata dictionaries, which are in turn used to obtain metadata for instance and/or subclass data nodes. This process has the following steps:

**1. [Generate Metadata Dictionaries](#generate-metadata-dictionaries):** In order to efficiently obtain metadata for all non-ontology instance and/or subclass data nodes and all relations, we first read in the data for each type (i.e. genes, rna, pathways, variants, and relations) and convert them into a dictionary. Then, each metadata dictionary is merged together and saved to a `master_metadata_dictionary`, keyed by identifier.
  - <u>Input Datasets</u>:  
    - Genes ➞ `Homo_sapiens.gene_info`   
    - RNA ➞ `ensembl_identifier_data_cleaned.txt` 
    - Pathways ➞ [`reactome2py API`](https://github.com/reactome/reactome2py) ; `ReactomePathways.txt`; `gene_association.reactome.gz`; `ChEBI2Reactome_All_Levels.txt`; `kegg_reactome.csv`   
    - Variants ➞ `variant_summary.txt`  
    - Relations ➞ `ro_with_imports.owl`  
    
Example Metadata Dictionary Output:

```python
{
    'nodes': {
        'http://www.ncbi.nlm.nih.gov/gene/1': {
            'Label': 'A1BG',
            'Description': "A1BG has locus group protein-coding' and is located on chromosome 19 (19q13.43).",
            'Synonym': 'HYST2477alpha-1B-glycoprotein|HEL-S-163pA|ABG|A1B|GAB'} ... },
    'relations': {
        'http://purl.obolibrary.org/obo/RO_0002533': {
            'Label': 'sequence atomic unit',
            'Description': 'Any individual unit of a collection of like units arranged in a linear order',
            'Synonym': 'None'} ... }
}
```

<br>

**2. [Write Metadata Files](#write-metadata-files):** The `master_metadata_dictionary` dictionary from _Step 1_ is `pickled` and saved to the `resources/node_data/` directory.

<br>

***

### Generate Metadata Dictionaries  <a class="anchor" id="generate-metadata-dictionaries"></a>
In this step, the goal is to create a metadata dictionary for each node type that does not rely on API data. In this case, only the **Gene**, **RNA**, and **Variant** nodes require data that is not from an API.


***

#### Genes Metadata Dictionary <a class="anchor" id="gene-metadata"></a>

The nested dictionary of gene metadata is created by looping over the merged data described in the prior column. The `keys` of the dictionary are `Entrez gene identifiers` and the `values` are dictionaries for each metadata type.

In [ ]:
# entrez gene data
entrez_gene_data = pandas.read_csv(unprocessed_data_location + 'Homo_sapiens.gene_info', header=0, delimiter='\t', low_memory=False)

# remove all rows that are not human
entrez_gene_data = entrez_gene_data.loc[entrez_gene_data['#tax_id'].apply(lambda x: x == 9606)]

# replace NaN and '-' with 'None'
entrez_gene_data.fillna('None', inplace=True)
entrez_gene_data.replace('-','None', inplace=True, regex=False)

In [ ]:
# create metadata
genes, lab, desc, syn = [], [], [], []
for idx, row in tqdm(entrez_gene_data.iterrows(), total=entrez_gene_data.shape[0]):
    gene_id, sym, defn, gene_type = row['GeneID'], row['Symbol'], row['description'], row['type_of_gene']
    chrom, map_loc, s1, s2 = row['chromosome'], row['map_location'], row['Synonyms'], row['Other_designations']
    if gene_id != 'None':
        genes.append('http://www.ncbi.nlm.nih.gov/gene/' + str(gene_id))
        if sym != 'None' or sym != '': lab.append(sym)
        else: lab.append('Entrez_ID:' + gene_id)
        if 'None' not in [defn, gene_type, chrom, map_loc]:
            desc_str = "{} has locus group '{}' and is located on chromosome {} ({})."
            desc.append(desc_str.format(sym, gene_type, chrom, map_loc))
        else: desc.append("{} locus group '{}'.".format(sym, gene_type))
        if s1 != 'None' and s2 != 'None': syn.append('|'.join(set([x for x in (s1 + s2).split('|') if x != 'None' or x != ''])))
        elif s1 != 'None': syn.append('|'.join(set([x for x in s1.split('|') if x != 'None' or x != ''])))
        elif s2 != 'None': syn.append('|'.join(set([x for x in s2.split('|') if x != 'None' or x != ''])))
        else: syn.append('None')

# combine into new data frame
metadata = pandas.DataFrame(list(zip(genes, lab, desc, syn)), columns=['ID', 'Label', 'Description', 'Synonym'])
metadata = metadata.astype(str)
metadata.drop_duplicates(subset='ID', keep='first', inplace=True)

# convert df to dictionary
metadata.set_index('ID', inplace=True)
gene_metadata_dict = metadata.to_dict('index')

***

#### RNA Metadata Dictionary  <a class="anchor" id="rna-metadata"></a>

The nested dictionary of rna metadata is created by looping over the cleaned human [Ensembl](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#ensembl) gene, RNA, and protein identifier data set (`ensembl_identifier_data_cleaned.txt`). The `keys` of the dictionary are `Ensembl transcript identifiers` and the `values` are dictionaries for each metadata type.

In [ ]:
# load data
rna_gene_data = pandas.read_csv(processed_data_location + 'ensembl_identifier_data_cleaned.txt', header=0, delimiter='\t', low_memory=False)

# remove rows without identifiers
rna_gene_data = rna_gene_data.loc[rna_gene_data['transcript_stable_id'].apply(lambda x: x != 'None')]

# remove unneeded columns
rna_gene_data.drop(['ensembl_gene_id', 'symbol', 'protein_stable_id', 'uniprot_id', 'master_transcript_type',
                    'entrez_id', 'ensembl_gene_type', 'master_gene_type', 'symbol'], axis=1, inplace=True)

# remove duplicates
rna_gene_data.drop_duplicates(subset=['transcript_stable_id', 'transcript_name', 'ensembl_transcript_type'], keep='first', inplace=True)

# replace NaN with 'None'
rna_gene_data.fillna('None', inplace=True)

In [ ]:
# create metadata
rna, lab, desc, syn = [], [], [], []
for idx, row in tqdm(rna_gene_data.iterrows(), total=rna_gene_data.shape[0]):
    rna_id, ent_type, nme = row['transcript_stable_id'], row['ensembl_transcript_type'], row['transcript_name']
    rna.append('https://uswest.ensembl.org/Homo_sapiens/Transcript/Summary?t=' + rna_id)
    if nme != 'None':
        lab.append(nme)
    else:
        lab.append('Ensembl_Transcript_ID:' + rna_id)
        nme = 'Ensembl_Transcript_ID:' + rna_id
    if ent_type != 'None': desc.append("Transcript {} is classified as type '{}'.".format(nme, ent_type))
    else: desc.append('None')
    syn.append('None')

# combine into new data frame
metadata = pandas.DataFrame(list(zip(rna, lab, desc, syn)), columns=['ID', 'Label', 'Description', 'Synonym'])
metadata = metadata.astype(str)
metadata.drop_duplicates(subset='ID', keep='first', inplace=True)

# convert df to dictionary
metadata.set_index('ID', inplace=True)
rna_metadata_dict = metadata.to_dict('index')

***

#### Variant Metadata Dictionary <a class="anchor" id="variant-metadata"></a>  

The nested dictionary of rna metadata is created by looping over the human [ClinVar Variant](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#clinvar) identifier data set (`variant_summary.txt`). The `keys` of the dictionary are `dbSNP identifiers` and the `values` are dictionaries for each metadata type.

In [ ]:
# download data
url = 'ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz'
if not os.path.exists(unprocessed_data_location + 'variant_summary.txt'):
    data_downloader(url, unprocessed_data_location)

# load data
var_data = pandas.read_csv(unprocessed_data_location + 'variant_summary.txt', header=0, delimiter='\t', low_memory=False)

# remove rows without identifiers
var_data = var_data.loc[var_data['Assembly'].apply(lambda x: x == 'GRCh38')]
var_data = var_data.loc[var_data['RS# (dbSNP)'].apply(lambda x: x != -1)]

# de-dup data
var_metadata = var_data[['#AlleleID', 'Type', 'Name', 'ClinicalSignificance', 'RS# (dbSNP)', 'Origin',
                         'ChromosomeAccession', 'Chromosome', 'Start', 'Stop', 'ReferenceAllele',
                         'Assembly', 'AlternateAllele','Cytogenetic', 'ReviewStatus', 'LastEvaluated']] 

# replace NaN with 'None'
var_metadata.replace('na', 'None', inplace=True)
var_metadata.fillna('None', inplace=True)

# remove duplicate dbSNP ids by choosing the most recent reviewed variant
var_metadata.sort_values('LastEvaluated', ascending=False, inplace=True)
var_metadata.drop_duplicates(subset='RS# (dbSNP)', keep='first', inplace=True)

In [ ]:
# create metadata
variant, label, desc, syn = [], [], [], []
for idx, row in tqdm(var_metadata.iterrows(), total=var_metadata.shape[0]):
    var_id, lab = row['RS# (dbSNP)'], row['Name']
    if var_id != 'None':
        variant.append('https://www.ncbi.nlm.nih.gov/snp/rs' + str(var_id))
        if lab != 'None': label.append(lab)
        else: label.append('dbSNP_ID:rs' + str(var_id))
        sent = "This variant is a {} {} located on chromosome {} ({}, start:{}/stop:{} positions, " +\
               "cytogenetic location:{}) and has clinical significance '{}'. " +\
               "This entry is for the {} and was last reviewed on {} with review status '{}'."
        desc.append(sent.format(row['Origin'].replace(';', '/'), row['Type'].replace(';', '/'), row['Chromosome'], row['ChromosomeAccession'],
                                row['Start'], row['Stop'], row['Cytogenetic'], row['ClinicalSignificance'],
                                row['Assembly'], row['LastEvaluated'], row['ReviewStatus']).replace('None', 'UNKNOWN'))
        syn.append('None')
    
# combine into new data frame
var_metadata_final = pandas.DataFrame(list(zip(variant, label, desc, syn)), columns =['ID', 'Label', 'Description', 'Synonym'])
var_metadata_final.drop_duplicates(subset=None, keep='first', inplace=True)
var_metadata_final = var_metadata_final.astype(str)

# convert df to dictionary
var_metadata_final.set_index('ID', inplace=True)
var_metadata_dict = var_metadata_final.to_dict('index') 

***

#### Pathway Metadata Dictionary <a class="anchor" id="pathway-metadata"></a>  

The nested dictionary of pathway metadata is created by looping over the human [Reactome Pathway Database](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#reactome-pathway-database) identifier data set (`ReactomePathways.txt`); Reactome-Gene Association data (`gene_association.reactome.gz`), and Reactome-ChEBI data (`ChEBI2Reactome_All_Levels.txt`). The `keys` of the dictionary are `Reactome identifiers` and the `values` are dictionaries for each metadata type.

In [ ]:
# download reactome pathways data
url = 'https://reactome.org/download/current/ReactomePathways.txt'
if not os.path.exists(unprocessed_data_location + 'ReactomePathways.txt'):
    data_downloader(url, unprocessed_data_location)
# load data
reactome_pathways = pandas.read_csv(unprocessed_data_location + 'ReactomePathways.txt', header=None, delimiter='\t', low_memory=False)
reactome_pathways = reactome_pathways.loc[reactome_pathways[2].apply(lambda x: x == 'Homo sapiens')] 

# reactome gene association data
url = 'https://reactome.org/download/current/gene_association.reactome.gz'
if not os.path.exists(unprocessed_data_location + 'gene_association.reactome'):
    data_downloader(url, unprocessed_data_location)
# load data
reactome_pathways2 = pandas.read_csv(unprocessed_data_location + 'gene_association.reactome', header=None, delimiter='\t', skiprows=3, low_memory=False)
reactome_pathways2 = reactome_pathways2.loc[reactome_pathways2[12].apply(lambda x: x == 'taxon:9606')]
reactome_pathways2[5].replace('REACTOME:','', inplace=True, regex=True) 

# reactome CHEBI data
url = 'https://reactome.org/download/current/ChEBI2Reactome_All_Levels.txt'
if not os.path.exists(unprocessed_data_location + 'ChEBI2Reactome_All_Levels.txt'):
    data_downloader(url, unprocessed_data_location)
# load data
reactome_pathways3 = pandas.read_csv(unprocessed_data_location + 'ChEBI2Reactome_All_Levels.txt', header=None, delimiter='\t', low_memory=False)
# remove all non-human pathways and save as list
reactome_pathways3 = reactome_pathways3.loc[reactome_pathways3[5].apply(lambda x: x == 'Homo sapiens')] 

In [ ]:
# get metadata
nodes = list(set(reactome_pathways[0]) | set(reactome_pathways2[5]) | set(reactome_pathways3[1]))
pathway_metadata_final = metadata_api_mapper(nodes)

# update dictionary
pathway_metadata_final['ID'] = pathway_metadata_final['ID'].map('https://reactome.org/content/detail/{}'.format)
pathway_metadata_final.set_index('ID', inplace=True)

# convert df to dictionary
pathway_metadata_dict = pathway_metadata_final.to_dict('index') 

***

#### Relations Metadata Dictionary <a class="anchor" id="relations-metadata"></a>  

The nested dictionary of relation metadata is created by looping over the human [Relations Ontology](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources#relations-ontology) identifier data set (`ro_with_imports.owl`). The `keys` of the dictionary are `Relations Ontology identifiers` and the `values` are dictionaries for each metadata type.

In [ ]:
# download ontology
if not os.path.exists(unprocessed_data_location + 'ro_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/ro.owl',
                             unprocessed_data_location + 'ro_with_imports.owl'))
# load graph
ro_graph = Graph().parse(unprocessed_data_location + 'ro_with_imports.owl')
print('There are {} edges in the ontology (date:{})'.format(len(ro_graph), datetime.datetime.now().strftime('%m/%d/%Y')))

In [ ]:
# get metadata
relation_metadata_dict, obo = {}, Namespace('http://purl.obolibrary.org/obo/')

# get ontology information
cls = [x for x in gets_ontology_classes(ro_graph) if '/RO_' in str(x)] +\
      [x for x in gets_object_properties(ro_graph) if '/RO_' in str(x)]
master_synonyms = [x for x in ro_graph if 'synonym' in str(x[1]).lower() and isinstance(x[0], URIRef)]

for x in tqdm(cls):
    # labels
    cls_label = [x for x in ro_graph.objects(x, RDFS.label) if '@' not in n3(x) or '@en' in n3(x)]
    labels = str(cls_label[0]) if len(cls_label) > 0 else 'None'
    # synonyms
    cls_syn = [str(i[2]) for i in master_synonyms if x == i[0]]
    synonym = str(cls_syn[0]) if len(cls_syn) > 0 else 'None'
    # description
    cls_desc = [x for x in ro_graph.objects(x, obo.IAO_0000115) if '@' not in n3(x) or '@en' in n3(x)]
    desc = '|'.join([str(cls_desc[0])]) if len(cls_desc) > 0 else 'None'
    
    relation_metadata_dict[str(x)] = {
        'Label': labels, 'Description': desc, 'Synonym': synonym
    }

***

**Create Master Metadata Dictionary** 

To make it easier to navigate the mapping of each instance node in an edge, a master dictionary is created and keyed by node type. This is most useful when both nodes in an edge are instances, but of different data types (e.g. `gene-rna`).


In [ ]:
# combine all metadata dictionaries
master_metadata_dictionary = {'nodes': {**gene_metadata_dict,
                                        **rna_metadata_dict,
                                        **var_metadata_dict,
                                        **pathway_metadata_dict},
                              'relations': relation_metadata_dict}

# verify metadata strings are properly formatted
temp_copy = master_metadata_dictionary.copy(); master_metadata_dictionary = dict()
for key, value in tqdm(temp_copy.items()):
    master_metadata_dictionary[key] = {}
    for ent_key, ent_value in value.items():
        updated_inner_dict = {k: re.sub('\s\s+', ' ', v.replace('\n', ' '))
                              if v is not None else v for k, v in ent_value.items()}
        master_metadata_dictionary[key][ent_key] = updated_inner_dict
del temp_copy

# save dictionary locally
pickle.dump(master_metadata_dictionary, open(node_data_location + 'node_metadata_dict.pkl', 'wb'), protocol=4)


<br>

***
***

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```